In [165]:
# PythonSim imports
from pythonSim import SimFunctions, SimClasses
from model_parameters import ModelParameters

from patients.patient import Patient
from patients.leukemia_patient import LeukemiaPatient
from patients.transplant_patient import TransplantPatient
from patients.other_patient import OtherPatient

from utils import assign_nurse_station

# Python package imports
import math
import pandas as pd
import numpy as np
import scipy.stats as scs

# Progress bar imports
from tqdm.notebook import tqdm_notebook
import time

In [166]:
# Initializing event calendar
Calendar = SimClasses.EventCalendar()

In [167]:
model_parameters = ModelParameters()

In [173]:
q_flow_station = SimClasses.Resource()
q_flow_station.SetUnits(1)
q_flow_station_queue = SimClasses.FIFOQueue()

secretary_station = SimClasses.Resource()
secretary_station.SetUnits(2)

secretary_station_queue = SimClasses.FIFOQueue()

nurse_station_1 = SimClasses.Resource()
nurse_station_1.SetUnits(1)
nurse_station_2 = SimClasses.Resource()
nurse_station_2.SetUnits(1)
nurse_station_3 = SimClasses.Resource()
nurse_station_3.SetUnits(1)
nurse_station_4 = SimClasses.Resource()
nurse_station_4.SetUnits(1)
nurse_station_5 = SimClasses.Resource()
nurse_station_5.SetUnits(1)
nurse_station_6 = SimClasses.Resource()
nurse_station_6.SetUnits(1)

nurse_station_1_queue = SimClasses.FIFOQueue()
nurse_station_2_queue = SimClasses.FIFOQueue()
nurse_station_3_queue = SimClasses.FIFOQueue()
nurse_station_4_queue = SimClasses.FIFOQueue()
nurse_station_5_queue = SimClasses.FIFOQueue()
nurse_station_6_queue = SimClasses.FIFOQueue()

leukemia_doctor_1 = SimClasses.Resource()
leukemia_doctor_1.SetUnits(1)
leukemia_doctor_2 = SimClasses.Resource()
leukemia_doctor_2.SetUnits(1)
transplant_doctor_1 = SimClasses.Resource()
transplant_doctor_1.SetUnits(1)
transplant_doctor_2 = SimClasses.Resource()
transplant_doctor_2.SetUnits(1)
transplant_doctor_3 = SimClasses.Resource()
transplant_doctor_3.SetUnits(1)

leukemia_doctor_1_queue = SimClasses.FIFOQueue()
leukemia_doctor_2_queue = SimClasses.FIFOQueue()
transplant_doctor_1_queue = SimClasses.FIFOQueue()
transplant_doctor_2_queue = SimClasses.FIFOQueue()
transplant_doctor_3_queue = SimClasses.FIFOQueue()

In [179]:
q_flow_station_wait_time = SimClasses.DTStat()

secretary_station_wait_time = SimClasses.DTStat()

nurse_station_1_wait_time = SimClasses.DTStat()
nurse_station_2_wait_time = SimClasses.DTStat()
nurse_station_3_wait_time = SimClasses.DTStat()
nurse_station_4_wait_time = SimClasses.DTStat()
nurse_station_5_wait_time = SimClasses.DTStat()
nurse_station_6_wait_time = SimClasses.DTStat()

leukemia_doctor_1_wait_time = SimClasses.DTStat()
leukemia_doctor_2_wait_time = SimClasses.DTStat()

transplant_doctor_1_wait_time = SimClasses.DTStat()
transplant_doctor_2_wait_time = SimClasses.DTStat()
transplant_doctor_3_wait_time = SimClasses.DTStat()

In [184]:
q_flow_station_wait_time_AVG = []

secretary_station_wait_time_AVG = []

nurse_station_1_wait_time_AVG = []
nurse_station_2_wait_time_AVG = []
nurse_station_3_wait_time_AVG = []
nurse_station_4_wait_time_AVG = []
nurse_station_5_wait_time_AVG = []
nurse_station_6_wait_time_AVG = []

leukemia_doctor_1_wait_time_AVG = []
leukemia_doctor_2_wait_time_AVG = []

transplant_doctor_1_wait_time_AVG = []
transplant_doctor_2_wait_time_AVG = []
transplant_doctor_3_wait_time_AVG = []

q_flow_station_queue_length_AVG = []

secretary_station_queue_length_AVG = []

nurse_station_1_queue_length_AVG = []
nurse_station_2_queue_length_AVG = []
nurse_station_3_queue_length_AVG = []
nurse_station_4_queue_length_AVG = []
nurse_station_5_queue_length_AVG = []
nurse_station_6_queue_length_AVG = []

leukemia_doctor_1_queue_length_AVG = []
leukemia_doctor_2_queue_length_AVG = []

transplant_doctor_1_queue_length_AVG = []
transplant_doctor_2_queue_length_AVG = []
transplant_doctor_3_queue_length_AVG = []

q_flow_station_busy_AVG = []

secretary_station_busy_AVG = []

nurse_station_1_busy_AVG = []
nurse_station_2_busy_AVG = []
nurse_station_3_busy_AVG = []
nurse_station_4_busy_AVG = []
nurse_station_5_busy_AVG = []
nurse_station_6_busy_AVG = []

leukemia_doctor_1_busy_AVG = []
leukemia_doctor_2_busy_AVG = []

transplant_doctor_1_busy_AVG = []
transplant_doctor_2_busy_AVG = []
transplant_doctor_3_busy_AVG = []

leukemia_total_process_time_AVG = []
transplant_total_process_time_AVG = []

In [ ]:
q_flow_station_queue_length = []

secretary_station_queue_length = []

nurse_station_1_queue_length = []
nurse_station_2_queue_length = []
nurse_station_3_queue_length = []
nurse_station_4_queue_length = []
nurse_station_5_queue_length = []
nurse_station_6_queue_length = []

leukemia_doctor_1_queue_length = []
leukemia_doctor_2_queue_length = []

transplant_doctor_1_queue_length = []
transplant_doctor_2_queue_length = []
transplant_doctor_3_queue_length = []

### We assume here that all patients begin arriving at 08:00 (time 0 is 07:00 hence why we add 60)
#### Currently we assume that they are all summoned at 20 minute intervals for simplicity

In [ ]:
leukemia_doctor_1_number_of_patients = model_parameters.leukemia_doctor_1_number_of_regular_patients + model_parameters.leukemia_doctor_1_number_of_complex_patients
leukemia_doctor_1_patient_arrival_times = [60 + i*20 for i in range(leukemia_doctor_1_number_of_patients)]
leukemia_doctor_2_number_of_patients = model_parameters.leukemia_doctor_2_number_of_regular_patients + model_parameters.leukemia_doctor_2_number_of_complex_patients
leukemia_doctor_2_patient_arrival_times = [60 + i*20 for i in range(leukemia_doctor_2_number_of_patients)]

transplant_doctor_1_number_of_patients = model_parameters.transplant_doctor_1_number_of_regular_patients + model_parameters.transplant_doctor_1_number_of_complex_patients
transplant_doctor_1_patient_arrival_times = [60 + i*20 for i in range(transplant_doctor_1_number_of_patients)]
transplant_doctor_2_number_of_patients = model_parameters.transplant_doctor_2_number_of_regular_patients + model_parameters.transplant_doctor_2_number_of_complex_patients
transplant_doctor_2_patient_arrival_times = [60 + i*20 for i in range(transplant_doctor_2_number_of_patients)]
transplant_doctor_3_number_of_patients = model_parameters.transplant_doctor_3_number_of_regular_patients + model_parameters.transplant_doctor_3_number_of_complex_patients
transplant_doctor_3_patient_arrival_times = [60 + i*20 for i in range(transplant_doctor_3_number_of_patients)]

other_patients_arrival_times = [60 + i*10 for i in range(model_parameters.number_of_other_patients)]

In [ ]:
def create_patient_schedule(arrival_time: int):
    return {'arrival_time': arrival_time,
            'doctor_consultation_time': arrival_time + 120}

def initialize_patients(patient_arrival_times: list[int],
                        doctor_name: str,
                        calendar: SimClasses.EventCalendar):
    leukemia = doctor_name.contains('leukemia')
    transplant = doctor_name.contains('transplant')
    patients = []
    if leukemia:
        probability_of_visiting_nurse = model_parameters.probability_of_visiting_nurse_leukemia

        if doctor_name.contains('1'):
            probability_of_complex_patient = model_parameters.leukemia_doctor_1_probability_of_complex_patient
        else:
            probability_of_complex_patient = model_parameters.leukemia_doctor_2_probability_of_complex_patient

        for arrival_time in patient_arrival_times:
            patients.append(LeukemiaPatient(create_patient_schedule(arrival_time),
                                            doctor_name,
                                            probability_of_complex_patient,
                                            probability_of_visiting_nurse))
    elif transplant:
        probability_of_visiting_nurse = model_parameters.probability_of_visiting_nurse_transplant
        if doctor_name.contains('1'):
            probability_of_complex_patient = model_parameters.transplant_doctor_1_probability_of_complex_patient
        elif doctor_name.contains('2'):
            probability_of_complex_patient = model_parameters.transplant_doctor_2_probability_of_complex_patient
        else:
            probability_of_complex_patient = model_parameters.transplant_doctor_3_probability_of_complex_patient

        for arrival_time in patient_arrival_times:
            patients.append(TransplantPatient(create_patient_schedule(arrival_time),
                                              doctor_name,
                                              probability_of_complex_patient,
                                              probability_of_visiting_nurse))
    else: #other
        probability_of_visiting_nurse = model_parameters.probability_of_visiting_nurse_other
        probability_of_complex_patient = model_parameters.probability_of_complex_other_patient

        for arrival_time in patient_arrival_times:
            patients.append(OtherPatient(create_patient_schedule(arrival_time),
                                         doctor_name,
                                         probability_of_complex_patient,
                                         probability_of_visiting_nurse))
    
    for patient in patients:
        patient.schedule_arrival(calendar)
        
    return patients

In [ ]:
leukemia_doctor_1_patients = initialize_patients(leukemia_doctor_1_patient_arrival_times, "leukemia_doctor_1")
leukemia_doctor_2_patients = initialize_patients(leukemia_doctor_2_patient_arrival_times, "leukemia_doctor_2")
transplant_doctor_1_patients = initialize_patients(transplant_doctor_1_patient_arrival_times, "transplant_doctor_1")
transplant_doctor_2_patients = initialize_patients(transplant_doctor_2_patient_arrival_times, "transplant_doctor_2")
transplant_doctor_3_patients = initialize_patients(transplant_doctor_3_patient_arrival_times, "transplant_doctor_3")
other_patients = initialize_patients(other_patients_arrival_times, "other")

In [191]:
def q_flow_start_of_waiting(new_patient : Patient):
    q_flow_station_queue.Add(new_patient)
    new_patient.enter_q_flow_queue()
    if q_flow_station.CurrentNumBusy < q_flow_station.NumberOfUnits:
        q_flow_station.Seize(1)
        next_patient = q_flow_station_queue.Remove()
        q_flow_station_wait_time.Record(SimClasses.Clock - next_patient.enter_q_flow_queue_time)
        SimFunctions.SchedulePlus(Calendar, "q_flow_station_service_start", 0, next_patient)

In [192]:
def q_flow_station_service(new_patient : Patient):
    q_flow_service_duration = np.random.exponential(model_parameters.q_flow_mean_service_time)
    SimFunctions.SchedulePlus(Calendar, "q_flow_station_service_end", q_flow_service_duration, new_patient)

In [ ]:
def q_flow_station_service_end(new_patient: Patient):
    new_patient.enter_secretary_queue_time = SimClasses.Clock
    SimFunctions.SchedulePlus(Calendar, "secretary_station_start_of_waiting", 0, new_patient)

    q_flow_station.Free(1)
    if q_flow_station_queue.NumQueue() > 0:
        next_patient = q_flow_station_queue.Remove()
        q_flow_station_wait_time.Record(SimClasses.Clock - next_patient.enter_q_flow_queue_time)
        SimFunctions.SchedulePlus(Calendar, "q_flow_station_service_start", 0, next_patient)

In [ ]:
def secretary_station_start_of_waiting(new_patient: Patient):
    secretary_station_queue.Add(new_patient)
    new_patient.enter_secretary_queue()
    if secretary_station.CurrentNumBusy < secretary_station.NumberOfUnits:
        secretary_station.Seize(1)
        next_patient = secretary_station_queue.Remove()
        secretary_station_wait_time.Record(SimClasses.Clock - next_patient.enter_secretary_queue_time)
        SimFunctions.SchedulePlus(Calendar, "secretary_station_service_start", 0, next_patient)

In [ ]:
def secretary_station_service_start(new_patient: Patient):
    secretary_station_service_duration = np.random.exponential(model_parameters.secretary_mean_service_time)
    SimFunctions.SchedulePlus(Calendar, "secretary_station_service_end", secretary_station_service_duration, new_patient)

In [ ]:
def secretary_station_service_end(new_patient: Patient):
    patient_type = new_patient.get_type()
    has_nurse_appointment = new_patient.visits_nurse
    if has_nurse_appointment:
        nurse_station = assign_nurse_station(patient_type, model_parameters)
        SimFunctions.SchedulePlus(Calendar, f'{nurse_station}_start_of_waiting', 0, new_patient)
    else:
        SimFunctions.SchedulePlus(Calendar, f'{new_patient.doctor_name}_start_of_waiting', 0, new_patient)
    
    secretary_station.Free(1)
    if secretary_station_queue.NumQueue() > 0:
        next_patient = secretary_station_queue.Remove()
        secretary_station_wait_time.Record(SimClasses.Clock - next_patient.enter_secretary_queue_time)
        SimFunctions.SchedulePlus(Calendar, "secretary_station_service_start", 0, next_patient)

In [ ]:
def nurse_station_1_start_of_waiting(new_patient: Patient):
    nurse_station_1_queue.Add(new_patient)
    new_patient.enter_nurse_queue()
    if nurse_station_1.CurrentNumBusy < nurse_station_1.NumberOfUnits:
        nurse_station_1.Seize(1)
        next_patient = nurse_station_1_queue.Remove()
        nurse_station_1_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_1_service_start", 0, next_patient)

def nurse_station_2_start_of_waiting(new_patient: Patient):
    nurse_station_2_queue.Add(new_patient)
    new_patient.enter_nurse_queue()
    if nurse_station_2.CurrentNumBusy < nurse_station_2.NumberOfUnits:
        nurse_station_2.Seize(1)
        next_patient = nurse_station_2_queue.Remove()
        nurse_station_2_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_2_service_start", 0, next_patient)

def nurse_station_3_start_of_waiting(new_patient: Patient):
    nurse_station_3_queue.Add(new_patient)
    new_patient.enter_nurse_queue()
    if nurse_station_3.CurrentNumBusy < nurse_station_3.NumberOfUnits:
        nurse_station_3.Seize(1)
        next_patient = nurse_station_3_queue.Remove()
        nurse_station_3_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_3_service_start", 0, next_patient)

def nurse_station_4_start_of_waiting(new_patient: Patient):
    nurse_station_4_queue.Add(new_patient)
    new_patient.enter_nurse_queue()
    if nurse_station_4.CurrentNumBusy < nurse_station_4.NumberOfUnits:
        nurse_station_4.Seize(1)
        next_patient = nurse_station_4_queue.Remove()
        nurse_station_4_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_4_service_start", 0, next_patient)

def nurse_station_5_start_of_waiting(new_patient: Patient):
    nurse_station_5_queue.Add(new_patient)
    new_patient.enter_nurse_queue()
    if nurse_station_5.CurrentNumBusy < nurse_station_5.NumberOfUnits:
        nurse_station_5.Seize(1)
        next_patient = nurse_station_5_queue.Remove()
        nurse_station_5_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_5_service_start", 0, next_patient)

def nurse_station_6_start_of_waiting(new_patient: Patient):
    nurse_station_6_queue.Add(new_patient)
    new_patient.enter_nurse_queue()
    if nurse_station_6.CurrentNumBusy < nurse_station_6.NumberOfUnits:
        nurse_station_6.Seize(1)
        next_patient = nurse_station_6_queue.Remove()
        nurse_station_6_wait_time.Record(SimClasses.Clock - next_patient.enter_nurse_queue_time)
        SimFunctions.SchedulePlus(Calendar, "nurse_station_6_service_start", 0, next_patient)

In [ ]:
def nurse_station_1_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level

    if patient_complexity == "regular":
        mean_service_time = model_parameters.nurse_mean_service_time_regular
    else:
        mean_service_time = model_parameters.nurse_mean_service_time_complex

    service_duration = np.random.exponential(mean_service_time)

    SimFunctions.SchedulePlus(Calendar, "nurse_station_1_service_end", service_duration, new_patient)

def nurse_station_2_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level

    if patient_complexity == "regular":
        mean_service_time = model_parameters.nurse_mean_service_time_regular
    else:
        mean_service_time = model_parameters.nurse_mean_service_time_complex

    service_duration = np.random.exponential(mean_service_time)

    SimFunctions.SchedulePlus(Calendar, "nurse_station_2_service_end", service_duration, new_patient)

def nurse_station_3_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level

    if patient_complexity == "regular":
        mean_service_time = model_parameters.nurse_mean_service_time_regular
    else:
        mean_service_time = model_parameters.nurse_mean_service_time_complex

    service_duration = np.random.exponential(mean_service_time)

    SimFunctions.SchedulePlus(Calendar, "nurse_station_3_service_end", service_duration, new_patient)

def nurse_station_4_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level

    if patient_complexity == "regular":
        mean_service_time = model_parameters.nurse_mean_service_time_regular
    else:
        mean_service_time = model_parameters.nurse_mean_service_time_complex

    service_duration = np.random.exponential(mean_service_time)

    SimFunctions.SchedulePlus(Calendar, "nurse_station_4_service_end", service_duration, new_patient)

def nurse_station_5_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level

    if patient_complexity == "regular":
        mean_service_time = model_parameters.nurse_mean_service_time_regular
    else:
        mean_service_time = model_parameters.nurse_mean_service_time_complex

    service_duration = np.random.exponential(mean_service_time)

    SimFunctions.SchedulePlus(Calendar, "nurse_station_5_service_end", service_duration, new_patient)

def nurse_station_6_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level

    if patient_complexity == "regular":
        mean_service_time = model_parameters.nurse_mean_service_time_regular
    else:
        mean_service_time = model_parameters.nurse_mean_service_time_complex

    service_duration = np.random.exponential(mean_service_time)

    SimFunctions.SchedulePlus(Calendar, "nurse_station_6_service_end", service_duration, new_patient)

In [ ]:
def nurse_station_1_service_end(new_patient: Patient):
    doctor_name = new_patient.doctor_name
    SimFunctions.SchedulePlus(Calendar, f"{doctor_name}_start_of_waiting", 0, new_patient)

def nurse_station_2_service_end(new_patient: Patient):
    doctor_name = new_patient.doctor_name
    SimFunctions.SchedulePlus(Calendar, f"{doctor_name}_start_of_waiting", 0, new_patient)

def nurse_station_3_service_end(new_patient: Patient):
    doctor_name = new_patient.doctor_name
    SimFunctions.SchedulePlus(Calendar, f"{doctor_name}_start_of_waiting", 0, new_patient)

def nurse_station_4_service_end(new_patient: Patient):
    doctor_name = new_patient.doctor_name
    SimFunctions.SchedulePlus(Calendar, f"{doctor_name}_start_of_waiting", 0, new_patient)

def nurse_station_5_service_end(new_patient: Patient):
    doctor_name = new_patient.doctor_name
    SimFunctions.SchedulePlus(Calendar, f"{doctor_name}_start_of_waiting", 0, new_patient)

def nurse_station_6_service_end(new_patient: Patient):
    doctor_name = new_patient.doctor_name
    SimFunctions.SchedulePlus(Calendar, f"{doctor_name}_start_of_waiting", 0, new_patient)

In [ ]:
def leukemia_doctor_1_start_of_waiting(new_patient: Patient):
    leukemia_doctor_1_queue.Add(new_patient)
    SimFunctions.SchedulePlus(Calendar, "leukemia_doctor_1_service_start", 0, new_patient)
    if leukemia_doctor_1.CurrentNumBusy < leukemia_doctor_1.NumberOfUnits:
        leukemia_doctor_1.Seize(1)
        next_patient = leukemia_doctor_1_queue.Remove()
        leukemia_doctor_1_wait_time.Record(SimClasses.Clock - next_patient.enter_leukemia_doctor_1_queue_time)
        SimFunctions.SchedulePlus(Calendar, "leukemia_doctor_1_service_start", 0, next_patient)

def leukemia_doctor_2_start_of_waiting(new_patient: Patient):
    leukemia_doctor_2_queue.Add(new_patient)
    SimFunctions.SchedulePlus(Calendar, "leukemia_doctor_2_service_start", 0, new_patient)
    if leukemia_doctor_2.CurrentNumBusy < leukemia_doctor_2.NumberOfUnits:
        leukemia_doctor_2.Seize(1)
        next_patient = leukemia_doctor_2_queue.Remove()
        leukemia_doctor_2_wait_time.Record(SimClasses.Clock - next_patient.enter_leukemia_doctor_2_queue_time)
        SimFunctions.SchedulePlus(Calendar, "leukemia_doctor_2_service_start", 0, next_patient)

def transplant_doctor_1_start_of_waiting(new_patient: Patient):
    transplant_doctor_1_queue.Add(new_patient)
    SimFunctions.SchedulePlus(Calendar, "transplant_doctor_1_service_start", 0, new_patient)
    if transplant_doctor_1.CurrentNumBusy < transplant_doctor_1.NumberOfUnits:
        transplant_doctor_1.Seize(1)
        next_patient = transplant_doctor_1_queue.Remove()
        transplant_doctor_1_wait_time.Record(SimClasses.Clock - next_patient.enter_transplant_doctor_1_queue_time)
        SimFunctions.SchedulePlus(Calendar, "transplant_doctor_1_service_start", 0, next_patient)

def transplant_doctor_2_start_of_waiting(new_patient: Patient):
    transplant_doctor_2_queue.Add(new_patient)
    SimFunctions.SchedulePlus(Calendar, "transplant_doctor_2_service_start", 0, new_patient)
    if transplant_doctor_2.CurrentNumBusy < transplant_doctor_2.NumberOfUnits:
        transplant_doctor_2.Seize(1)
        next_patient = transplant_doctor_2_queue.Remove()
        transplant_doctor_2_wait_time.Record(SimClasses.Clock - next_patient.enter_transplant_doctor_2_queue_time)
        SimFunctions.SchedulePlus(Calendar, "transplant_doctor_2_service_start", 0, next_patient)

def transplant_doctor_3_start_of_waiting(new_patient: Patient):
    transplant_doctor_3_queue.Add(new_patient)
    SimFunctions.SchedulePlus(Calendar, "transplant_doctor_3_service_start", 0, new_patient)
    if transplant_doctor_3.CurrentNumBusy < transplant_doctor_3.NumberOfUnits:
        transplant_doctor_3.Seize(1)
        next_patient = transplant_doctor_3_queue.Remove()
        transplant_doctor_3_wait_time.Record(SimClasses.Clock - next_patient.enter_transplant_doctor_3_queue_time)
        SimFunctions.SchedulePlus(Calendar, "transplant_doctor_3_service_start", 0, next_patient)

def other_doctor_start_of_waiting(new_patient: Patient):
    #other patients doctor treatment is not included in the model
    SimFunctions.SchedulePlus(Calendar, "process_complete", 0, new_patient)

In [ ]:
def leukemia_doctor_1_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level
    if patient_complexity == "regular":
        mean_service_time = model_parameters.leukemia_doctor_1_mean_service_time_regular
    else:
        mean_service_time = model_parameters.leukemia_doctor_1_mean_service_time_complex
    service_duration = np.random.exponential(mean_service_time)
    SimFunctions.SchedulePlus(Calendar, "leukemia_doctor_1_service_end", service_duration, new_patient)

def leukemia_doctor_2_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level
    if patient_complexity == "regular":
        mean_service_time = model_parameters.leukemia_doctor_2_mean_service_time_regular
    else:
        mean_service_time = model_parameters.leukemia_doctor_2_mean_service_time_complex
    service_duration = np.random.exponential(mean_service_time)
    SimFunctions.SchedulePlus(Calendar, "leukemia_doctor_2_service_end", service_duration, new_patient)

def transplant_doctor_1_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level
    if patient_complexity == "regular":
        mean_service_time = model_parameters.transplant_doctor_1_mean_service_time_regular
    else:
        mean_service_time = model_parameters.transplant_doctor_1_mean_service_time_complex
    service_duration = np.random.exponential(mean_service_time)
    SimFunctions.SchedulePlus(Calendar, "transplant_doctor_1_service_end", service_duration, new_patient)

def transplant_doctor_2_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level
    if patient_complexity == "regular":
        mean_service_time = model_parameters.transplant_doctor_2_mean_service_time_regular
    else:
        mean_service_time = model_parameters.transplant_doctor_2_mean_service_time_complex
    service_duration = np.random.exponential(mean_service_time)
    SimFunctions.SchedulePlus(Calendar, "transplant_doctor_2_service_end", service_duration, new_patient)

def transplant_doctor_3_service_start(new_patient: Patient):
    patient_complexity = new_patient.complexity_level
    if patient_complexity == "regular":
        mean_service_time = model_parameters.transplant_doctor_3_mean_service_time_regular
    else:
        mean_service_time = model_parameters.transplant_doctor_3_mean_service_time_complex
    service_duration = np.random.exponential(mean_service_time)
    SimFunctions.SchedulePlus(Calendar, "transplant_doctor_3_service_end", service_duration, new_patient)

In [ ]:
def leukemia_doctor_1_service_end(new_patient: Patient):
    SimFunctions.SchedulePlus(Calendar, "process_complete", 0, new_patient)

def leukemia_doctor_2_service_end(new_patient: Patient):
    SimFunctions.SchedulePlus(Calendar, "process_complete", 0, new_patient)

def transplant_doctor_1_service_end(new_patient: Patient):
    SimFunctions.SchedulePlus(Calendar, "process_complete", 0, new_patient)

def transplant_doctor_2_service_end(new_patient: Patient):
    SimFunctions.SchedulePlus(Calendar, "process_complete", 0, new_patient)

def transplant_doctor_3_service_end(new_patient: Patient):
    SimFunctions.SchedulePlus(Calendar, "process_complete", 0, new_patient)

In [193]:
def BrooklynVerbalEvaluationEnd(NewImmigrant : ImmigrantClass.Immigrant()):
    
    if ((NewImmigrant.getAge() == "Child" or NewImmigrant.getAge() == "Old") and NewImmigrant.getEnglishProficiency() == "Fluent") or (NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "Fluent" and NewImmigrant.getEducationLevel() == "Bachelor"):
        SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NewImmigrant)
    
    elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "Fluent" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
        NewImmigrant.JoinCareerCoachingQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'BrooklynCareerCoachingStartOfWaiting', 0, NewImmigrant)
    
    elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelA":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'BrooklynCourseChildLevelAStartOfWaiting', 0, NewImmigrant)
    
    elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelB":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'BrooklynCourseChildLevelBStartOfWaiting', 0, NewImmigrant)
    
    elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelC":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'BrooklynCourseChildLevelCStartOfWaiting', 0, NewImmigrant)
    
    elif (NewImmigrant.getAge() == "Adult" or NewImmigrant.getAge() == "Old") and NewImmigrant.getEnglishProficiency() == "LevelA":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'BrooklynCourseAdultLevelAStartOfWaiting', 0, NewImmigrant)
    
    elif (NewImmigrant.getAge() == "Adult" or NewImmigrant.getAge() == "Old") and NewImmigrant.getEnglishProficiency() == "LevelB":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'BrooklynCourseAdultLevelBStartOfWaiting', 0, NewImmigrant)
    
    elif (NewImmigrant.getAge() == "Adult" or NewImmigrant.getAge() == "Old") and NewImmigrant.getEnglishProficiency() == "LevelC":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'BrooklynCourseAdultLevelCStartOfWaiting', 0, NewImmigrant)
        
    BrooklynLanguageAssessors.Free(1)
    if BrooklynQueueLanguageAssessors.NumQueue() > 0:
        NextImmigrant = BrooklynQueueLanguageAssessors.Remove()
        BrooklynLanguageAssessors.Seize(1)
        BrooklynLanguageAssessorsWaitTime.Record(SimClasses.Clock - NextImmigrant.JoinVerbalEvaluationQueueTime)
        SimFunctions.SchedulePlus(Calendar, "BrooklynVerbalEvaluationStart", 0, NextImmigrant)

In [194]:
def BrooklynCourseChildLevelAStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    BrooklynQueueTeachersChildLevelA.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if BrooklynTeachersChildLevelA.CurrentNumBusy < BrooklynTeachersChildLevelA.NumberOfUnits and BrooklynQueueTeachersChildLevelA.NumQueue() >= ClassSize:
        BrooklynTeachersChildLevelA.Seize(1)
        for i in range(ClassSize):
            Student = BrooklynQueueTeachersChildLevelA.Remove()
            BrooklynTeachersChildLevelAWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BrooklynCourseChildLevelAStart", 0, Student)

In [195]:
def BrooklynCourseChildLevelAStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "BrooklynCourseChildLevelAEnd", LanguageCourseDurationLevelA, Student)

In [196]:
def BrooklynCourseChildLevelAEnd(NextStudent):
    BrooklynTeachersChildLevelA.Free(1)
    SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
    if BrooklynTeachersChildLevelA.CurrentNumBusy < BrooklynTeachersChildLevelA.NumberOfUnits and BrooklynQueueTeachersChildLevelA.NumQueue() >= ClassSize:
        BrooklynTeachersChildLevelA.Seize(1)
        for i in range(ClassSize):
            Student = BrooklynQueueTeachersChildLevelA.Remove()
            BrooklynTeachersChildLevelAWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BrooklynCourseChildLevelAStart", 0, Student)

In [197]:
def BrooklynCourseChildLevelBStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    BrooklynQueueTeachersChildLevelB.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if BrooklynTeachersChildLevelB.CurrentNumBusy < BrooklynTeachersChildLevelB.NumberOfUnits and BrooklynQueueTeachersChildLevelB.NumQueue() >= ClassSize:
        BrooklynTeachersChildLevelB.Seize(1)
        for i in range(ClassSize):
            Student = BrooklynQueueTeachersChildLevelB.Remove()
            BrooklynTeachersChildLevelBWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BrooklynCourseChildLevelBStart", 0, Student)

In [198]:
def BrooklynCourseChildLevelBStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "BrooklynCourseChildLevelBEnd", LanguageCourseDurationLevelB, Student)

In [199]:
def BrooklynCourseChildLevelBEnd(NextStudent):
    BrooklynTeachersChildLevelB.Free(1)
    SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
    if BrooklynTeachersChildLevelB.CurrentNumBusy < BrooklynTeachersChildLevelB.NumberOfUnits and BrooklynQueueTeachersChildLevelB.NumQueue() >= ClassSize:
        BrooklynTeachersChildLevelB.Seize(1)
        for i in range(ClassSize):
            Student = BrooklynQueueTeachersChildLevelB.Remove()
            BrooklynTeachersChildLevelBWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BrooklynCourseChildLevelBStart", 0, Student)

In [200]:
def BrooklynCourseChildLevelCStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    BrooklynQueueTeachersChildLevelC.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if BrooklynTeachersChildLevelC.CurrentNumBusy < BrooklynTeachersChildLevelC.NumberOfUnits and BrooklynQueueTeachersChildLevelC.NumQueue() >= ClassSize:
        BrooklynTeachersChildLevelC.Seize(1)
        for i in range(ClassSize):
            Student = BrooklynQueueTeachersChildLevelC.Remove()
            BrooklynTeachersChildLevelCWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BrooklynCourseChildLevelCStart", 0, Student)

In [201]:
def BrooklynCourseChildLevelCStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "BrooklynCourseChildLevelCEnd", LanguageCourseDurationLevelC, Student)

In [202]:
def BrooklynCourseChildLevelCEnd(NextStudent):
    BrooklynTeachersChildLevelC.Free(1)
    SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
    if BrooklynTeachersChildLevelC.CurrentNumBusy < BrooklynTeachersChildLevelC.NumberOfUnits and BrooklynQueueTeachersChildLevelC.NumQueue() >= ClassSize:
        BrooklynTeachersChildLevelC.Seize(1)
        for i in range(ClassSize):
            Student = BrooklynQueueTeachersChildLevelC.Remove()
            BrooklynTeachersChildLevelCWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BrooklynCourseChildLevelCStart", 0, Student)

In [203]:
def BrooklynCourseAdultLevelAStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    BrooklynQueueTeachersAdultLevelA.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if BrooklynTeachersAdultLevelA.CurrentNumBusy < BrooklynTeachersAdultLevelA.NumberOfUnits and BrooklynQueueTeachersAdultLevelA.NumQueue() >= ClassSize:
        BrooklynTeachersAdultLevelA.Seize(1)
        for i in range(ClassSize):
            Student = BrooklynQueueTeachersAdultLevelA.Remove()
            BrooklynTeachersAdultLevelAWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BrooklynCourseAdultLevelAStart", 0, Student)

In [204]:
def BrooklynCourseAdultLevelAStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "BrooklynCourseAdultLevelAEnd", LanguageCourseDurationLevelA, Student)

In [205]:
def BrooklynCourseAdultLevelAEnd(NextStudent):
    BrooklynTeachersAdultLevelA.Free(1)
    if NextStudent.getEducationLevel() == "Less Than Bachelor":
        NextStudent.JoinCareerCoachingQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'BrooklynCareerCoachingStartOfWaiting', 0, NextStudent)
    else:
        SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
        
    if BrooklynTeachersAdultLevelA.CurrentNumBusy < BrooklynTeachersAdultLevelA.NumberOfUnits and BrooklynQueueTeachersAdultLevelA.NumQueue() >= ClassSize:
        BrooklynTeachersAdultLevelA.Seize(1)
        for i in range(ClassSize):
            Student = BrooklynQueueTeachersAdultLevelA.Remove()
            BrooklynTeachersAdultLevelAWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BrooklynCourseAdultLevelAStart", 0, Student)

In [206]:
def BrooklynCourseAdultLevelBStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    BrooklynQueueTeachersAdultLevelB.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if BrooklynTeachersAdultLevelB.CurrentNumBusy < BrooklynTeachersAdultLevelB.NumberOfUnits and BrooklynQueueTeachersAdultLevelB.NumQueue() >= ClassSize:
        BrooklynTeachersAdultLevelB.Seize(1)
        for i in range(ClassSize):
            Student = BrooklynQueueTeachersAdultLevelB.Remove()
            BrooklynTeachersAdultLevelBWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BrooklynCourseAdultLevelBStart", 0, Student)

In [207]:
def BrooklynCourseAdultLevelBStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "BrooklynCourseAdultLevelBEnd", LanguageCourseDurationLevelB, Student)

In [208]:
def BrooklynCourseAdultLevelBEnd(NextStudent):
    BrooklynTeachersAdultLevelB.Free(1)
    if NextStudent.getEducationLevel() == "Less Than Bachelor":
        NextStudent.JoinCareerCoachingQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'BrooklynCareerCoachingStartOfWaiting', 0, NextStudent)
    else:
        SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
        
    if BrooklynTeachersAdultLevelB.CurrentNumBusy < BrooklynTeachersAdultLevelB.NumberOfUnits and BrooklynQueueTeachersAdultLevelB.NumQueue() >= ClassSize:
        BrooklynTeachersAdultLevelB.Seize(1)
        for i in range(ClassSize):
            Student = BrooklynQueueTeachersAdultLevelB.Remove()
            BrooklynTeachersAdultLevelBWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BrooklynCourseAdultLevelBStart", 0, Student)

In [209]:
def BrooklynCourseAdultLevelCStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    BrooklynQueueTeachersAdultLevelC.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if BrooklynTeachersAdultLevelC.CurrentNumBusy < BrooklynTeachersAdultLevelC.NumberOfUnits and BrooklynQueueTeachersAdultLevelC.NumQueue() >= ClassSize:
        BrooklynTeachersAdultLevelC.Seize(1)
        for i in range(ClassSize):
            Student = BrooklynQueueTeachersAdultLevelC.Remove()
            BrooklynTeachersAdultLevelCWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BrooklynCourseAdultLevelCStart", 0, Student)

In [210]:
def BrooklynCourseAdultLevelCStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "BrooklynCourseAdultLevelCEnd", LanguageCourseDurationLevelC, Student)

In [211]:
def BrooklynCourseAdultLevelCEnd(NextStudent):
    BrooklynTeachersAdultLevelC.Free(1)
    if NextStudent.getEducationLevel() == "Less Than Bachelor":
        NextStudent.JoinCareerCoachingQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'BrooklynCareerCoachingStartOfWaiting', 0, NextStudent)
    else:
        SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
        
    if BrooklynTeachersAdultLevelC.CurrentNumBusy < BrooklynTeachersAdultLevelC.NumberOfUnits and BrooklynQueueTeachersAdultLevelC.NumQueue() >= ClassSize:
        BrooklynTeachersAdultLevelC.Seize(1)
        for i in range(ClassSize):
            Student = BrooklynQueueTeachersAdultLevelC.Remove()
            BrooklynTeachersAdultLevelCWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BrooklynCourseAdultLevelCStart", 0, Student)

In [212]:
def BrooklynCareerCoachingStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    BrooklynQueueCareerCoach.Add(NewImmigrant)
    NewImmigrant.EnterCareerCoachingQueue()
    if BrooklynCareerCoach.CurrentNumBusy < BrooklynCareerCoach.NumberOfUnits and BrooklynQueueCareerCoach.NumQueue() > 0:
        BrooklynCareerCoach.Seize(1)
        NextImmigrant = BrooklynQueueCareerCoach.Remove()
        BrooklynCareerCoachWaitTime.Record(SimClasses.Clock - NextImmigrant.getJoinCareerCoachingQueueTime())
        SimFunctions.SchedulePlus(Calendar, "BrooklynCareerCoachingStart", 0, NextImmigrant)

In [213]:
def BrooklynCareerCoachingStart(NewImmigrant : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "BrooklynCareerCoachingEnd", CareerCoachingSessionDuration, NewImmigrant)

In [214]:
def BrooklynCareerCoachingEnd(NewImmigrant : ImmigrantClass.Immigrant()):
    BrooklynCareerCoach.Free(1)
    SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NewImmigrant)
    if BrooklynCareerCoach.CurrentNumBusy < BrooklynCareerCoach.NumberOfUnits and BrooklynQueueCareerCoach.NumQueue() > 0:
        BrooklynCareerCoach.Seize(1)
        NextImmigrant = BrooklynQueueCareerCoach.Remove()
        BrooklynCareerCoachWaitTime.Record(SimClasses.Clock - NextImmigrant.getJoinCareerCoachingQueueTime())
        SimFunctions.SchedulePlus(Calendar, "BrooklynCareerCoachingStart", 0, NextImmigrant)

In [215]:
def ManhattanVerbalEvaluationStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    ManhattanQueueLanguageAssessors.Add(NewImmigrant)
    NewImmigrant.EnterVerbalEvaluationQueue()
    if ManhattanLanguageAssessors.CurrentNumBusy < ManhattanLanguageAssessors.NumberOfUnits:
        ManhattanLanguageAssessors.Seize(1)
        NextImmigrant = ManhattanQueueLanguageAssessors.Remove()
        ManhattanLanguageAssessorsWaitTime.Record(SimClasses.Clock - NextImmigrant.JoinVerbalEvaluationQueueTime)
        SimFunctions.SchedulePlus(Calendar, "ManhattanVerbalEvaluationStart", 0, NextImmigrant)

In [216]:
def ManhattanVerbalEvaluationStart(NewImmigrant : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "ManhattanVerbalEvaluationEnd", LanguageAssessmentDuration, NewImmigrant)

In [217]:
def ManhattanVerbalEvaluationEnd(NewImmigrant : ImmigrantClass.Immigrant()):
    if ((NewImmigrant.getAge() == "Child" or NewImmigrant.getAge() == "Old") and NewImmigrant.getEnglishProficiency() == "Fluent") or (NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "Fluent" and NewImmigrant.getEducationLevel() == "Bachelor"):
        SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NewImmigrant)
    
    elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "Fluent" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
        NewImmigrant.JoinCareerCoachingQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'ManhattanCareerCoachingStartOfWaiting', 0, NewImmigrant)
    
    elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelA":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'ManhattanCourseChildLevelAStartOfWaiting', 0, NewImmigrant)
    
    elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelB":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'ManhattanCourseChildLevelBStartOfWaiting', 0, NewImmigrant)
    
    elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelC":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'ManhattanCourseChildLevelCStartOfWaiting', 0, NewImmigrant)
    
    elif (NewImmigrant.getAge() == "Adult" or NewImmigrant.getAge() == "Old") and NewImmigrant.getEnglishProficiency() == "LevelA":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'ManhattanCourseAdultLevelAStartOfWaiting', 0, NewImmigrant)
    
    elif (NewImmigrant.getAge() == "Adult" or NewImmigrant.getAge() == "Old") and NewImmigrant.getEnglishProficiency() == "LevelB":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'ManhattanCourseAdultLevelBStartOfWaiting', 0, NewImmigrant)
    
    elif (NewImmigrant.getAge() == "Adult" or NewImmigrant.getAge() == "Old") and NewImmigrant.getEnglishProficiency() == "LevelC":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'ManhattanCourseAdultLevelCStartOfWaiting', 0, NewImmigrant)
        
    ManhattanLanguageAssessors.Free(1)
    if ManhattanQueueLanguageAssessors.NumQueue() > 0:
        NextImmigrant = ManhattanQueueLanguageAssessors.Remove()
        ManhattanLanguageAssessors.Seize(1)
        ManhattanLanguageAssessorsWaitTime.Record(SimClasses.Clock - NextImmigrant.JoinVerbalEvaluationQueueTime)
        SimFunctions.SchedulePlus(Calendar, "ManhattanVerbalEvaluationStart", 0, NextImmigrant)

In [218]:
def ManhattanCourseChildLevelAStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    ManhattanQueueTeachersChildLevelA.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if ManhattanTeachersChildLevelA.CurrentNumBusy < ManhattanTeachersChildLevelA.NumberOfUnits and ManhattanQueueTeachersChildLevelA.NumQueue() >= ClassSize:
        ManhattanTeachersChildLevelA.Seize(1)
        for i in range(ClassSize):
            Student = ManhattanQueueTeachersChildLevelA.Remove()
            ManhattanTeachersChildLevelAWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "ManhattanCourseChildLevelAStart", 0, Student)

In [219]:
def ManhattanCourseChildLevelAStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "ManhattanCourseChildLevelAEnd", LanguageCourseDurationLevelA, Student)

In [220]:
def ManhattanCourseChildLevelAEnd(NextStudent):
    ManhattanTeachersChildLevelA.Free(1)
    SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
    if ManhattanTeachersChildLevelA.CurrentNumBusy < ManhattanTeachersChildLevelA.NumberOfUnits and ManhattanQueueTeachersChildLevelA.NumQueue() >= ClassSize:
        ManhattanTeachersChildLevelA.Seize(1)
        for i in range(ClassSize):
            Student = ManhattanQueueTeachersChildLevelA.Remove()
            ManhattanTeachersChildLevelAWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "ManhattanCourseChildLevelAStart", 0, Student)

In [221]:
def ManhattanCourseChildLevelBStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    ManhattanQueueTeachersChildLevelB.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if ManhattanTeachersChildLevelB.CurrentNumBusy < ManhattanTeachersChildLevelB.NumberOfUnits and ManhattanQueueTeachersChildLevelB.NumQueue() >= ClassSize:
        ManhattanTeachersChildLevelB.Seize(1)
        for i in range(ClassSize):
            Student = ManhattanQueueTeachersChildLevelB.Remove()
            ManhattanTeachersChildLevelBWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "ManhattanCourseChildLevelBStart", 0, Student)

In [222]:
def ManhattanCourseChildLevelBStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "ManhattanCourseChildLevelBEnd", LanguageCourseDurationLevelB, Student)

In [223]:
def ManhattanCourseChildLevelBEnd(NextStudent):
    ManhattanTeachersChildLevelB.Free(1)
    SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
    if ManhattanTeachersChildLevelB.CurrentNumBusy < ManhattanTeachersChildLevelB.NumberOfUnits and ManhattanQueueTeachersChildLevelB.NumQueue() >= ClassSize:
        ManhattanTeachersChildLevelB.Seize(1)
        for i in range(ClassSize):
            Student = ManhattanQueueTeachersChildLevelB.Remove()
            ManhattanTeachersChildLevelBWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "ManhattanCourseChildLevelBStart", 0, Student)

In [224]:
def ManhattanCourseChildLevelCStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    ManhattanQueueTeachersChildLevelC.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if ManhattanTeachersChildLevelC.CurrentNumBusy < ManhattanTeachersChildLevelC.NumberOfUnits and ManhattanQueueTeachersChildLevelC.NumQueue() >= ClassSize:
        ManhattanTeachersChildLevelC.Seize(1)
        for i in range(ClassSize):
            Student = ManhattanQueueTeachersChildLevelC.Remove()
            ManhattanTeachersChildLevelCWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "ManhattanCourseChildLevelCStart", 0, Student)

In [225]:
def ManhattanCourseChildLevelCStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "ManhattanCourseChildLevelCEnd", LanguageCourseDurationLevelC, Student)

In [226]:
def ManhattanCourseChildLevelCEnd(NextStudent):
    ManhattanTeachersChildLevelC.Free(1)
    SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
    if ManhattanTeachersChildLevelC.CurrentNumBusy < ManhattanTeachersChildLevelC.NumberOfUnits and ManhattanQueueTeachersChildLevelC.NumQueue() >= ClassSize:
        ManhattanTeachersChildLevelC.Seize(1)
        for i in range(ClassSize):
            Student = ManhattanQueueTeachersChildLevelC.Remove()
            ManhattanTeachersChildLevelCWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "ManhattanCourseChildLevelCStart", 0, Student)

In [227]:
def ManhattanCourseAdultLevelAStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    ManhattanQueueTeachersAdultLevelA.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if ManhattanTeachersAdultLevelA.CurrentNumBusy < ManhattanTeachersAdultLevelA.NumberOfUnits and ManhattanQueueTeachersAdultLevelA.NumQueue() >= ClassSize:
        ManhattanTeachersAdultLevelA.Seize(1)
        for i in range(ClassSize):
            Student = ManhattanQueueTeachersAdultLevelA.Remove()
            ManhattanTeachersAdultLevelAWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "ManhattanCourseAdultLevelAStart", 0, Student)

In [228]:
def ManhattanCourseAdultLevelAStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "ManhattanCourseAdultLevelAEnd", LanguageCourseDurationLevelA, Student)

In [229]:
def ManhattanCourseAdultLevelAEnd(NextStudent):
    ManhattanTeachersAdultLevelA.Free(1)
    if NextStudent.getEducationLevel() == "Less Than Bachelor":
        NextStudent.JoinCareerCoachingQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'ManhattanCareerCoachingStartOfWaiting', 0, NextStudent)
    else:
        SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
        
    if ManhattanTeachersAdultLevelA.CurrentNumBusy < ManhattanTeachersAdultLevelA.NumberOfUnits and ManhattanQueueTeachersAdultLevelA.NumQueue() >= ClassSize:
        ManhattanTeachersAdultLevelA.Seize(1)
        for i in range(ClassSize):
            Student = ManhattanQueueTeachersAdultLevelA.Remove()
            ManhattanTeachersAdultLevelAWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "ManhattanCourseAdultLevelAStart", 0, Student)

In [230]:
def ManhattanCourseAdultLevelBStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    ManhattanQueueTeachersAdultLevelB.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if ManhattanTeachersAdultLevelB.CurrentNumBusy < ManhattanTeachersAdultLevelB.NumberOfUnits and ManhattanQueueTeachersAdultLevelB.NumQueue() >= ClassSize:
        ManhattanTeachersAdultLevelB.Seize(1)
        for i in range(ClassSize):
            Student = ManhattanQueueTeachersAdultLevelB.Remove()
            ManhattanTeachersAdultLevelBWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "ManhattanCourseAdultLevelBStart", 0, Student)

In [231]:
def ManhattanCourseAdultLevelBStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "ManhattanCourseAdultLevelBEnd", LanguageCourseDurationLevelB, Student)

In [232]:
def ManhattanCourseAdultLevelBEnd(NextStudent):
    ManhattanTeachersAdultLevelB.Free(1)
    if NextStudent.getEducationLevel() == "Less Than Bachelor":
        NextStudent.JoinCareerCoachingQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'ManhattanCareerCoachingStartOfWaiting', 0, NextStudent)
    else:
        SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
        
    if ManhattanTeachersAdultLevelB.CurrentNumBusy < ManhattanTeachersAdultLevelB.NumberOfUnits and ManhattanQueueTeachersAdultLevelB.NumQueue() >= ClassSize:
        ManhattanTeachersAdultLevelB.Seize(1)
        for i in range(ClassSize):
            Student = ManhattanQueueTeachersAdultLevelB.Remove()
            ManhattanTeachersAdultLevelBWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "ManhattanCourseAdultLevelBStart", 0, Student)

In [233]:
def ManhattanCourseAdultLevelCStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    ManhattanQueueTeachersAdultLevelC.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if ManhattanTeachersAdultLevelC.CurrentNumBusy < ManhattanTeachersAdultLevelC.NumberOfUnits and ManhattanQueueTeachersAdultLevelC.NumQueue() >= ClassSize:
        ManhattanTeachersAdultLevelC.Seize(1)
        for i in range(ClassSize):
            Student = ManhattanQueueTeachersAdultLevelC.Remove()
            ManhattanTeachersAdultLevelCWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "ManhattanCourseAdultLevelCStart", 0, Student)

In [234]:
def ManhattanCourseAdultLevelCStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "ManhattanCourseAdultLevelCEnd", LanguageCourseDurationLevelC, Student)

In [235]:
def ManhattanCourseAdultLevelCEnd(NextStudent):
    ManhattanTeachersAdultLevelC.Free(1)
    if NextStudent.getEducationLevel() == "Less Than Bachelor":
        NextStudent.JoinCareerCoachingQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'ManhattanCareerCoachingStartOfWaiting', 0, NextStudent)
    else:
        SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
        
    if ManhattanTeachersAdultLevelC.CurrentNumBusy < ManhattanTeachersAdultLevelC.NumberOfUnits and ManhattanQueueTeachersAdultLevelC.NumQueue() >= ClassSize:
        ManhattanTeachersAdultLevelC.Seize(1)
        for i in range(ClassSize):
            Student = ManhattanQueueTeachersAdultLevelC.Remove()
            ManhattanTeachersAdultLevelCWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "ManhattanCourseAdultLevelCStart", 0, Student)

In [236]:
def ManhattanCareerCoachingStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    ManhattanQueueCareerCoach.Add(NewImmigrant)
    NewImmigrant.EnterCareerCoachingQueue()
    if ManhattanCareerCoach.CurrentNumBusy < ManhattanCareerCoach.NumberOfUnits and ManhattanQueueCareerCoach.NumQueue() > 0:
        ManhattanCareerCoach.Seize(1)
        NextImmigrant = ManhattanQueueCareerCoach.Remove()
        ManhattanCareerCoachWaitTime.Record(SimClasses.Clock - NextImmigrant.getJoinCareerCoachingQueueTime())
        SimFunctions.SchedulePlus(Calendar, "ManhattanCareerCoachingStart", 0, NextImmigrant)

In [237]:
def ManhattanCareerCoachingStart(NewImmigrant : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "ManhattanCareerCoachingEnd", CareerCoachingSessionDuration, NewImmigrant)

In [238]:
def ManhattanCareerCoachingEnd(NewImmigrant : ImmigrantClass.Immigrant()):
    ManhattanCareerCoach.Free(1)
    SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NewImmigrant)
    if ManhattanCareerCoach.CurrentNumBusy < ManhattanCareerCoach.NumberOfUnits and ManhattanQueueCareerCoach.NumQueue() > 0:
        ManhattanCareerCoach.Seize(1)
        NextImmigrant = ManhattanQueueCareerCoach.Remove()
        ManhattanCareerCoachWaitTime.Record(SimClasses.Clock - NextImmigrant.getJoinCareerCoachingQueueTime())
        SimFunctions.SchedulePlus(Calendar, "ManhattanCareerCoachingStart", 0, NextImmigrant)

In [239]:
def QueensVerbalEvaluationStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    QueensQueueLanguageAssessors.Add(NewImmigrant)
    NewImmigrant.EnterVerbalEvaluationQueue()
    if QueensLanguageAssessors.CurrentNumBusy < QueensLanguageAssessors.NumberOfUnits:
        QueensLanguageAssessors.Seize(1)
        NextImmigrant = QueensQueueLanguageAssessors.Remove()
        QueensLanguageAssessorsWaitTime.Record(SimClasses.Clock - NextImmigrant.JoinVerbalEvaluationQueueTime)
        SimFunctions.SchedulePlus(Calendar, "QueensVerbalEvaluationStart", 0, NextImmigrant)

In [240]:
def QueensVerbalEvaluationStart(NewImmigrant : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "QueensVerbalEvaluationEnd", LanguageAssessmentDuration, NewImmigrant)

In [241]:
def QueensVerbalEvaluationEnd(NewImmigrant : ImmigrantClass.Immigrant()):
    if ((NewImmigrant.getAge() == "Child" or NewImmigrant.getAge() == "Old") and NewImmigrant.getEnglishProficiency() == "Fluent") or (NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "Fluent" and NewImmigrant.getEducationLevel() == "Bachelor"):
        SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NewImmigrant)
    
    elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "Fluent" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
        NewImmigrant.JoinCareerCoachingQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'QueensCareerCoachingStartOfWaiting', 0, NewImmigrant)
    
    elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelA":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'QueensCourseChildLevelAStartOfWaiting', 0, NewImmigrant)
    
    elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelB":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'QueensCourseChildLevelBStartOfWaiting', 0, NewImmigrant)
    
    elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelC":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'QueensCourseChildLevelCStartOfWaiting', 0, NewImmigrant)
    
    elif (NewImmigrant.getAge() == "Adult" or NewImmigrant.getAge() == "Old") and NewImmigrant.getEnglishProficiency() == "LevelA":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'QueensCourseAdultLevelAStartOfWaiting', 0, NewImmigrant)
    
    elif (NewImmigrant.getAge() == "Adult" or NewImmigrant.getAge() == "Old") and NewImmigrant.getEnglishProficiency() == "LevelB":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'QueensCourseAdultLevelBStartOfWaiting', 0, NewImmigrant)
    
    elif (NewImmigrant.getAge() == "Adult" or NewImmigrant.getAge() == "Old") and NewImmigrant.getEnglishProficiency() == "LevelC":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'QueensCourseAdultLevelCStartOfWaiting', 0, NewImmigrant)
        
    QueensLanguageAssessors.Free(1)
    if QueensQueueLanguageAssessors.NumQueue() > 0:
        NextImmigrant = QueensQueueLanguageAssessors.Remove()
        QueensLanguageAssessors.Seize(1)
        QueensLanguageAssessorsWaitTime.Record(SimClasses.Clock - NextImmigrant.JoinVerbalEvaluationQueueTime)
        SimFunctions.SchedulePlus(Calendar, "QueensVerbalEvaluationStart", 0, NextImmigrant)

In [242]:
def QueensCourseChildLevelAStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    QueensQueueTeachersChildLevelA.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if QueensTeachersChildLevelA.CurrentNumBusy < QueensTeachersChildLevelA.NumberOfUnits and QueensQueueTeachersChildLevelA.NumQueue() >= ClassSize:
        QueensTeachersChildLevelA.Seize(1)
        for i in range(ClassSize):
            Student = QueensQueueTeachersChildLevelA.Remove()
            QueensTeachersChildLevelAWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "QueensCourseChildLevelAStart", 0, Student)

In [243]:
def QueensCourseChildLevelAStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "QueensCourseChildLevelAEnd", LanguageCourseDurationLevelA, Student)

In [244]:
def QueensCourseChildLevelAEnd(NextStudent):
    QueensTeachersChildLevelA.Free(1)
    SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
    if QueensTeachersChildLevelA.CurrentNumBusy < QueensTeachersChildLevelA.NumberOfUnits and QueensQueueTeachersChildLevelA.NumQueue() >= ClassSize:
        QueensTeachersChildLevelA.Seize(1)
        for i in range(ClassSize):
            Student = QueensQueueTeachersChildLevelA.Remove()
            QueensTeachersChildLevelAWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "QueensCourseChildLevelAStart", 0, Student)

In [245]:
def QueensCourseChildLevelBStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    QueensQueueTeachersChildLevelB.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if QueensTeachersChildLevelB.CurrentNumBusy < QueensTeachersChildLevelB.NumberOfUnits and QueensQueueTeachersChildLevelB.NumQueue() >= ClassSize:
        QueensTeachersChildLevelB.Seize(1)
        for i in range(ClassSize):
            Student = QueensQueueTeachersChildLevelB.Remove()
            QueensTeachersChildLevelBWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "QueensCourseChildLevelBStart", 0, Student)

In [246]:
def QueensCourseChildLevelBStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "QueensCourseChildLevelBEnd", LanguageCourseDurationLevelB, Student)

In [247]:
def QueensCourseChildLevelBEnd(NextStudent):
    QueensTeachersChildLevelB.Free(1)
    SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
    if QueensTeachersChildLevelB.CurrentNumBusy < QueensTeachersChildLevelB.NumberOfUnits and QueensQueueTeachersChildLevelB.NumQueue() >= ClassSize:
        QueensTeachersChildLevelB.Seize(1)
        for i in range(ClassSize):
            Student = QueensQueueTeachersChildLevelB.Remove()
            QueensTeachersChildLevelBWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "QueensCourseChildLevelBStart", 0, Student)

In [248]:
def QueensCourseChildLevelCStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    QueensQueueTeachersChildLevelC.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if QueensTeachersChildLevelC.CurrentNumBusy < QueensTeachersChildLevelC.NumberOfUnits and QueensQueueTeachersChildLevelC.NumQueue() >= ClassSize:
        QueensTeachersChildLevelC.Seize(1)
        for i in range(ClassSize):
            Student = QueensQueueTeachersChildLevelC.Remove()
            QueensTeachersChildLevelCWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "QueensCourseChildLevelCStart", 0, Student)

In [249]:
def QueensCourseChildLevelCStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "QueensCourseChildLevelCEnd", LanguageCourseDurationLevelC, Student)

In [250]:
def QueensCourseChildLevelCEnd(NextStudent):
    QueensTeachersChildLevelC.Free(1)
    SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
    if QueensTeachersChildLevelC.CurrentNumBusy < QueensTeachersChildLevelC.NumberOfUnits and QueensQueueTeachersChildLevelC.NumQueue() >= ClassSize:
        QueensTeachersChildLevelC.Seize(1)
        for i in range(ClassSize):
            Student = QueensQueueTeachersChildLevelC.Remove()
            QueensTeachersChildLevelCWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "QueensCourseChildLevelCStart", 0, Student)

In [251]:
def QueensCourseAdultLevelAStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    QueensQueueTeachersAdultLevelA.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if QueensTeachersAdultLevelA.CurrentNumBusy < QueensTeachersAdultLevelA.NumberOfUnits and QueensQueueTeachersAdultLevelA.NumQueue() >= ClassSize:
        QueensTeachersAdultLevelA.Seize(1)
        for i in range(ClassSize):
            Student = QueensQueueTeachersAdultLevelA.Remove()
            QueensTeachersAdultLevelAWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "QueensCourseAdultLevelAStart", 0, Student)

In [252]:
def QueensCourseAdultLevelAStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "QueensCourseAdultLevelAEnd", LanguageCourseDurationLevelA, Student)

In [253]:
def QueensCourseAdultLevelAEnd(NextStudent):
    QueensTeachersAdultLevelA.Free(1)
    if NextStudent.getEducationLevel() == "Less Than Bachelor":
        NextStudent.JoinCareerCoachingQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'QueensCareerCoachingStartOfWaiting', 0, NextStudent)
    else:
        SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
        
    if QueensTeachersAdultLevelA.CurrentNumBusy < QueensTeachersAdultLevelA.NumberOfUnits and QueensQueueTeachersAdultLevelA.NumQueue() >= ClassSize:
        QueensTeachersAdultLevelA.Seize(1)
        for i in range(ClassSize):
            Student = QueensQueueTeachersAdultLevelA.Remove()
            QueensTeachersAdultLevelAWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "QueensCourseAdultLevelAStart", 0, Student)

In [254]:
def QueensCourseAdultLevelBStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    QueensQueueTeachersAdultLevelB.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if QueensTeachersAdultLevelB.CurrentNumBusy < QueensTeachersAdultLevelB.NumberOfUnits and QueensQueueTeachersAdultLevelB.NumQueue() >= ClassSize:
        QueensTeachersAdultLevelB.Seize(1)
        for i in range(ClassSize):
            Student = QueensQueueTeachersAdultLevelB.Remove()
            QueensTeachersAdultLevelBWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "QueensCourseAdultLevelBStart", 0, Student)

In [255]:
def QueensCourseAdultLevelBStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "QueensCourseAdultLevelBEnd", LanguageCourseDurationLevelB, Student)

In [256]:
def QueensCourseAdultLevelBEnd(NextStudent):
    QueensTeachersAdultLevelB.Free(1)
    if NextStudent.getEducationLevel() == "Less Than Bachelor":
        NextStudent.JoinCareerCoachingQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'QueensCareerCoachingStartOfWaiting', 0, NextStudent)
    else:
        SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
        
    if QueensTeachersAdultLevelB.CurrentNumBusy < QueensTeachersAdultLevelB.NumberOfUnits and QueensQueueTeachersAdultLevelB.NumQueue() >= ClassSize:
        QueensTeachersAdultLevelB.Seize(1)
        for i in range(ClassSize):
            Student = QueensQueueTeachersAdultLevelB.Remove()
            QueensTeachersAdultLevelBWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "QueensCourseAdultLevelBStart", 0, Student)

In [257]:
def QueensCourseAdultLevelCStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    QueensQueueTeachersAdultLevelC.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if QueensTeachersAdultLevelC.CurrentNumBusy < QueensTeachersAdultLevelC.NumberOfUnits and QueensQueueTeachersAdultLevelC.NumQueue() >= ClassSize:
        QueensTeachersAdultLevelC.Seize(1)
        for i in range(ClassSize):
            Student = QueensQueueTeachersAdultLevelC.Remove()
            QueensTeachersAdultLevelCWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "QueensCourseAdultLevelCStart", 0, Student)

In [258]:
def QueensCourseAdultLevelCStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "QueensCourseAdultLevelCEnd", LanguageCourseDurationLevelC, Student)

In [259]:
def QueensCourseAdultLevelCEnd(NextStudent):
    QueensTeachersAdultLevelC.Free(1)
    if NextStudent.getEducationLevel() == "Less Than Bachelor":
        NextStudent.JoinCareerCoachingQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'QueensCareerCoachingStartOfWaiting', 0, NextStudent)
    else:
        SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
        
    if QueensTeachersAdultLevelC.CurrentNumBusy < QueensTeachersAdultLevelC.NumberOfUnits and QueensQueueTeachersAdultLevelC.NumQueue() >= ClassSize:
        QueensTeachersAdultLevelC.Seize(1)
        for i in range(ClassSize):
            Student = QueensQueueTeachersAdultLevelC.Remove()
            QueensTeachersAdultLevelCWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "QueensCourseAdultLevelCStart", 0, Student)

In [260]:
def QueensCareerCoachingStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    QueensQueueCareerCoach.Add(NewImmigrant)
    NewImmigrant.EnterCareerCoachingQueue()
    if QueensCareerCoach.CurrentNumBusy < QueensCareerCoach.NumberOfUnits and QueensQueueCareerCoach.NumQueue() > 0:
        QueensCareerCoach.Seize(1)
        NextImmigrant = QueensQueueCareerCoach.Remove()
        QueensCareerCoachWaitTime.Record(SimClasses.Clock - NextImmigrant.getJoinCareerCoachingQueueTime())
        SimFunctions.SchedulePlus(Calendar, "QueensCareerCoachingStart", 0, NextImmigrant)

In [261]:
def QueensCareerCoachingStart(NewImmigrant : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "QueensCareerCoachingEnd", CareerCoachingSessionDuration, NewImmigrant)

In [262]:
def QueensCareerCoachingEnd(NewImmigrant : ImmigrantClass.Immigrant()):
    QueensCareerCoach.Free(1)
    SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NewImmigrant)
    if QueensCareerCoach.CurrentNumBusy < QueensCareerCoach.NumberOfUnits and QueensQueueCareerCoach.NumQueue() > 0:
        QueensCareerCoach.Seize(1)
        NextImmigrant = QueensQueueCareerCoach.Remove()
        QueensCareerCoachWaitTime.Record(SimClasses.Clock - NextImmigrant.getJoinCareerCoachingQueueTime())
        SimFunctions.SchedulePlus(Calendar, "QueensCareerCoachingStart", 0, NextImmigrant)

In [263]:
def BronxVerbalEvaluationStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    BronxQueueLanguageAssessors.Add(NewImmigrant)
    NewImmigrant.EnterVerbalEvaluationQueue()
    if BronxLanguageAssessors.CurrentNumBusy < BronxLanguageAssessors.NumberOfUnits:
        BronxLanguageAssessors.Seize(1)
        NextImmigrant = BronxQueueLanguageAssessors.Remove()
        BronxLanguageAssessorsWaitTime.Record(SimClasses.Clock - NextImmigrant.JoinVerbalEvaluationQueueTime)
        SimFunctions.SchedulePlus(Calendar, "BronxVerbalEvaluationStart", 0, NextImmigrant)

In [264]:
def BronxVerbalEvaluationStart(NewImmigrant : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "BronxVerbalEvaluationEnd", LanguageAssessmentDuration, NewImmigrant)

In [265]:
def BronxVerbalEvaluationEnd(NewImmigrant : ImmigrantClass.Immigrant()):
    if ((NewImmigrant.getAge() == "Child" or NewImmigrant.getAge() == "Old") and NewImmigrant.getEnglishProficiency() == "Fluent") or (NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "Fluent" and NewImmigrant.getEducationLevel() == "Bachelor"):
        SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NewImmigrant)
    
    elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "Fluent" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
        NewImmigrant.JoinCareerCoachingQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'BronxCareerCoachingStartOfWaiting', 0, NewImmigrant)
    
    elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelA":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'BronxCourseChildLevelAStartOfWaiting', 0, NewImmigrant)
    
    elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelB":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'BronxCourseChildLevelBStartOfWaiting', 0, NewImmigrant)
    
    elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelC":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'BronxCourseChildLevelCStartOfWaiting', 0, NewImmigrant)
    
    elif (NewImmigrant.getAge() == "Adult" or NewImmigrant.getAge() == "Old") and NewImmigrant.getEnglishProficiency() == "LevelA":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'BronxCourseAdultLevelAStartOfWaiting', 0, NewImmigrant)
    
    elif (NewImmigrant.getAge() == "Adult" or NewImmigrant.getAge() == "Old") and NewImmigrant.getEnglishProficiency() == "LevelB":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'BronxCourseAdultLevelBStartOfWaiting', 0, NewImmigrant)
    
    elif (NewImmigrant.getAge() == "Adult" or NewImmigrant.getAge() == "Old") and NewImmigrant.getEnglishProficiency() == "LevelC":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'BronxCourseAdultLevelCStartOfWaiting', 0, NewImmigrant)
        
    BronxLanguageAssessors.Free(1)
    if BronxQueueLanguageAssessors.NumQueue() > 0:
        NextImmigrant = BronxQueueLanguageAssessors.Remove()
        BronxLanguageAssessors.Seize(1)
        BronxLanguageAssessorsWaitTime.Record(SimClasses.Clock - NextImmigrant.JoinVerbalEvaluationQueueTime)
        SimFunctions.SchedulePlus(Calendar, "BronxVerbalEvaluationStart", 0, NextImmigrant)

In [266]:
def BronxCourseChildLevelAStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    BronxQueueTeachersChildLevelA.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if BronxTeachersChildLevelA.CurrentNumBusy < BronxTeachersChildLevelA.NumberOfUnits and BronxQueueTeachersChildLevelA.NumQueue() >= ClassSize:
        BronxTeachersChildLevelA.Seize(1)
        for i in range(ClassSize):
            Student = BronxQueueTeachersChildLevelA.Remove()
            BronxTeachersChildLevelAWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BronxCourseChildLevelAStart", 0, Student)

In [267]:
def BronxCourseChildLevelAStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "BronxCourseChildLevelAEnd", LanguageCourseDurationLevelA, Student)

In [268]:
def BronxCourseChildLevelAEnd(NextStudent):
    BronxTeachersChildLevelA.Free(1)
    SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
    if BronxTeachersChildLevelA.CurrentNumBusy < BronxTeachersChildLevelA.NumberOfUnits and BronxQueueTeachersChildLevelA.NumQueue() >= ClassSize:
        BronxTeachersChildLevelA.Seize(1)
        for i in range(ClassSize):
            Student = BronxQueueTeachersChildLevelA.Remove()
            BronxTeachersChildLevelAWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BronxCourseChildLevelAStart", 0, Student)

In [269]:
def BronxCourseChildLevelBStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    BronxQueueTeachersChildLevelB.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if BronxTeachersChildLevelB.CurrentNumBusy < BronxTeachersChildLevelB.NumberOfUnits and BronxQueueTeachersChildLevelB.NumQueue() >= ClassSize:
        BronxTeachersChildLevelB.Seize(1)
        for i in range(ClassSize):
            Student = BronxQueueTeachersChildLevelB.Remove()
            BronxTeachersChildLevelBWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BronxCourseChildLevelBStart", 0, Student)

In [270]:
def BronxCourseChildLevelBStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "BronxCourseChildLevelBEnd", LanguageCourseDurationLevelB, Student)

In [271]:
def BronxCourseChildLevelBEnd(NextStudent):
    BronxTeachersChildLevelB.Free(1)
    SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
    if BronxTeachersChildLevelB.CurrentNumBusy < BronxTeachersChildLevelB.NumberOfUnits and BronxQueueTeachersChildLevelB.NumQueue() >= ClassSize:
        BronxTeachersChildLevelB.Seize(1)
        for i in range(ClassSize):
            Student = BronxQueueTeachersChildLevelB.Remove()
            BronxTeachersChildLevelBWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BronxCourseChildLevelBStart", 0, Student)

In [272]:
def BronxCourseChildLevelCStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    BronxQueueTeachersChildLevelC.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if BronxTeachersChildLevelC.CurrentNumBusy < BronxTeachersChildLevelC.NumberOfUnits and BronxQueueTeachersChildLevelC.NumQueue() >= ClassSize:
        BronxTeachersChildLevelC.Seize(1)
        for i in range(ClassSize):
            Student = BronxQueueTeachersChildLevelC.Remove()
            BronxTeachersChildLevelCWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BronxCourseChildLevelCStart", 0, Student)

In [273]:
def BronxCourseChildLevelCStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "BronxCourseChildLevelCEnd", LanguageCourseDurationLevelC, Student)

In [274]:
def BronxCourseChildLevelCEnd(NextStudent):
    BronxTeachersChildLevelC.Free(1)
    SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
    if BronxTeachersChildLevelC.CurrentNumBusy < BronxTeachersChildLevelC.NumberOfUnits and BronxQueueTeachersChildLevelC.NumQueue() >= ClassSize:
        BronxTeachersChildLevelC.Seize(1)
        for i in range(ClassSize):
            Student = BronxQueueTeachersChildLevelC.Remove()
            BronxTeachersChildLevelCWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BronxCourseChildLevelCStart", 0, Student)

In [275]:
def BronxCourseAdultLevelAStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    BronxQueueTeachersAdultLevelA.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if BronxTeachersAdultLevelA.CurrentNumBusy < BronxTeachersAdultLevelA.NumberOfUnits and BronxQueueTeachersAdultLevelA.NumQueue() >= ClassSize:
        BronxTeachersAdultLevelA.Seize(1)
        for i in range(ClassSize):
            Student = BronxQueueTeachersAdultLevelA.Remove()
            BronxTeachersAdultLevelAWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BronxCourseAdultLevelAStart", 0, Student)

In [276]:
def BronxCourseAdultLevelAStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "BronxCourseAdultLevelAEnd", LanguageCourseDurationLevelA, Student)

In [277]:
def BronxCourseAdultLevelAEnd(NextStudent):
    BronxTeachersAdultLevelA.Free(1)
    if NextStudent.getEducationLevel() == "Less Than Bachelor":
        NextStudent.JoinCareerCoachingQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'BronxCareerCoachingStartOfWaiting', 0, NextStudent)
    else:
        SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
        
    if BronxTeachersAdultLevelA.CurrentNumBusy < BronxTeachersAdultLevelA.NumberOfUnits and BronxQueueTeachersAdultLevelA.NumQueue() >= ClassSize:
        BronxTeachersAdultLevelA.Seize(1)
        for i in range(ClassSize):
            Student = BronxQueueTeachersAdultLevelA.Remove()
            BronxTeachersAdultLevelAWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BronxCourseAdultLevelAStart", 0, Student)

In [278]:
def BronxCourseAdultLevelBStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    BronxQueueTeachersAdultLevelB.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if BronxTeachersAdultLevelB.CurrentNumBusy < BronxTeachersAdultLevelB.NumberOfUnits and BronxQueueTeachersAdultLevelB.NumQueue() >= ClassSize:
        BronxTeachersAdultLevelB.Seize(1)
        for i in range(ClassSize):
            Student = BronxQueueTeachersAdultLevelB.Remove()
            BronxTeachersAdultLevelBWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BronxCourseAdultLevelBStart", 0, Student)

In [279]:
def BronxCourseAdultLevelBStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "BronxCourseAdultLevelBEnd", LanguageCourseDurationLevelB, Student)

In [280]:
def BronxCourseAdultLevelBEnd(NextStudent):
    BronxTeachersAdultLevelB.Free(1)
    if NextStudent.getEducationLevel() == "Less Than Bachelor":
        NextStudent.JoinCareerCoachingQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'BronxCareerCoachingStartOfWaiting', 0, NextStudent)
    else:
        SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
        
    if BronxTeachersAdultLevelB.CurrentNumBusy < BronxTeachersAdultLevelB.NumberOfUnits and BronxQueueTeachersAdultLevelB.NumQueue() >= ClassSize:
        BronxTeachersAdultLevelB.Seize(1)
        for i in range(ClassSize):
            Student = BronxQueueTeachersAdultLevelB.Remove()
            BronxTeachersAdultLevelBWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BronxCourseAdultLevelBStart", 0, Student)

In [281]:
def BronxCourseAdultLevelCStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    BronxQueueTeachersAdultLevelC.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if BronxTeachersAdultLevelC.CurrentNumBusy < BronxTeachersAdultLevelC.NumberOfUnits and BronxQueueTeachersAdultLevelC.NumQueue() >= ClassSize:
        BronxTeachersAdultLevelC.Seize(1)
        for i in range(ClassSize):
            Student = BronxQueueTeachersAdultLevelC.Remove()
            BronxTeachersAdultLevelCWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BronxCourseAdultLevelCStart", 0, Student)

In [282]:
def BronxCourseAdultLevelCStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "BronxCourseAdultLevelCEnd", LanguageCourseDurationLevelC, Student)

In [283]:
def BronxCourseAdultLevelCEnd(NextStudent):
    BronxTeachersAdultLevelC.Free(1)
    if NextStudent.getEducationLevel() == "Less Than Bachelor":
        NextStudent.JoinCareerCoachingQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'BronxCareerCoachingStartOfWaiting', 0, NextStudent)
    else:
        SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
        
    if BronxTeachersAdultLevelC.CurrentNumBusy < BronxTeachersAdultLevelC.NumberOfUnits and BronxQueueTeachersAdultLevelC.NumQueue() >= ClassSize:
        BronxTeachersAdultLevelC.Seize(1)
        for i in range(ClassSize):
            Student = BronxQueueTeachersAdultLevelC.Remove()
            BronxTeachersAdultLevelCWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "BronxCourseAdultLevelCStart", 0, Student)

In [284]:
def BronxCareerCoachingStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    BronxQueueCareerCoach.Add(NewImmigrant)
    NewImmigrant.EnterCareerCoachingQueue()
    if BronxCareerCoach.CurrentNumBusy < BronxCareerCoach.NumberOfUnits and BronxQueueCareerCoach.NumQueue() > 0:
        BronxCareerCoach.Seize(1)
        NextImmigrant = BronxQueueCareerCoach.Remove()
        BronxCareerCoachWaitTime.Record(SimClasses.Clock - NextImmigrant.getJoinCareerCoachingQueueTime())
        SimFunctions.SchedulePlus(Calendar, "BronxCareerCoachingStart", 0, NextImmigrant)

In [285]:
def BronxCareerCoachingStart(NewImmigrant : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "BronxCareerCoachingEnd", CareerCoachingSessionDuration, NewImmigrant)

In [286]:
def BronxCareerCoachingEnd(NewImmigrant : ImmigrantClass.Immigrant()):
    BronxCareerCoach.Free(1)
    SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NewImmigrant)
    if BronxCareerCoach.CurrentNumBusy < BronxCareerCoach.NumberOfUnits and BronxQueueCareerCoach.NumQueue() > 0:
        BronxCareerCoach.Seize(1)
        NextImmigrant = BronxQueueCareerCoach.Remove()
        BronxCareerCoachWaitTime.Record(SimClasses.Clock - NextImmigrant.getJoinCareerCoachingQueueTime())
        SimFunctions.SchedulePlus(Calendar, "BronxCareerCoachingStart", 0, NextImmigrant)

In [287]:
def StatenIslandVerbalEvaluationStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    StatenIslandQueueLanguageAssessors.Add(NewImmigrant)
    NewImmigrant.EnterVerbalEvaluationQueue()
    if StatenIslandLanguageAssessors.CurrentNumBusy < StatenIslandLanguageAssessors.NumberOfUnits:
        StatenIslandLanguageAssessors.Seize(1)
        NextImmigrant = StatenIslandQueueLanguageAssessors.Remove()
        StatenIslandLanguageAssessorsWaitTime.Record(SimClasses.Clock - NextImmigrant.JoinVerbalEvaluationQueueTime)
        SimFunctions.SchedulePlus(Calendar, "StatenIslandVerbalEvaluationStart", 0, NextImmigrant)

In [288]:
def StatenIslandVerbalEvaluationStart(NewImmigrant : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "StatenIslandVerbalEvaluationEnd", LanguageAssessmentDuration, NewImmigrant)

In [289]:
def StatenIslandVerbalEvaluationEnd(NewImmigrant : ImmigrantClass.Immigrant()):
    if ((NewImmigrant.getAge() == "Child" or NewImmigrant.getAge() == "Old") and NewImmigrant.getEnglishProficiency() == "Fluent") or (NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "Fluent" and NewImmigrant.getEducationLevel() == "Bachelor"):
        SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NewImmigrant)
    
    elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "Fluent" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
        NewImmigrant.JoinCareerCoachingQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'StatenIslandCareerCoachingStartOfWaiting', 0, NewImmigrant)
    
    elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelA":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'StatenIslandCourseChildLevelAStartOfWaiting', 0, NewImmigrant)
    
    elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelB":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'StatenIslandCourseChildLevelBStartOfWaiting', 0, NewImmigrant)
    
    elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelC":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'StatenIslandCourseChildLevelCStartOfWaiting', 0, NewImmigrant)
    
    elif (NewImmigrant.getAge() == "Adult" or NewImmigrant.getAge() == "Old") and NewImmigrant.getEnglishProficiency() == "LevelA":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'StatenIslandCourseAdultLevelAStartOfWaiting', 0, NewImmigrant)
    
    elif (NewImmigrant.getAge() == "Adult" or NewImmigrant.getAge() == "Old") and NewImmigrant.getEnglishProficiency() == "LevelB":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'StatenIslandCourseAdultLevelBStartOfWaiting', 0, NewImmigrant)
    
    elif (NewImmigrant.getAge() == "Adult" or NewImmigrant.getAge() == "Old") and NewImmigrant.getEnglishProficiency() == "LevelC":
        NewImmigrant.JoinClassQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'StatenIslandCourseAdultLevelCStartOfWaiting', 0, NewImmigrant)
        
    StatenIslandLanguageAssessors.Free(1)
    if StatenIslandQueueLanguageAssessors.NumQueue() > 0:
        NextImmigrant = StatenIslandQueueLanguageAssessors.Remove()
        StatenIslandLanguageAssessors.Seize(1)
        StatenIslandLanguageAssessorsWaitTime.Record(SimClasses.Clock - NextImmigrant.JoinVerbalEvaluationQueueTime)
        SimFunctions.SchedulePlus(Calendar, "StatenIslandVerbalEvaluationStart", 0, NextImmigrant)

In [290]:
def StatenIslandCourseChildLevelAStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    StatenIslandQueueTeachersChildLevelA.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if StatenIslandTeachersChildLevelA.CurrentNumBusy < StatenIslandTeachersChildLevelA.NumberOfUnits and StatenIslandQueueTeachersChildLevelA.NumQueue() >= ClassSize:
        StatenIslandTeachersChildLevelA.Seize(1)
        for i in range(ClassSize):
            Student = StatenIslandQueueTeachersChildLevelA.Remove()
            StatenIslandTeachersChildLevelAWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "StatenIslandCourseChildLevelAStart", 0, Student)

In [291]:
def StatenIslandCourseChildLevelAStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "StatenIslandCourseChildLevelAEnd", LanguageCourseDurationLevelA, Student)

In [292]:
def StatenIslandCourseChildLevelAEnd(NextStudent):
    StatenIslandTeachersChildLevelA.Free(1)
    SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
    if StatenIslandTeachersChildLevelA.CurrentNumBusy < StatenIslandTeachersChildLevelA.NumberOfUnits and StatenIslandQueueTeachersChildLevelA.NumQueue() >= ClassSize:
        StatenIslandTeachersChildLevelA.Seize(1)
        for i in range(ClassSize):
            Student = StatenIslandQueueTeachersChildLevelA.Remove()
            StatenIslandTeachersChildLevelAWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "StatenIslandCourseChildLevelAStart", 0, Student)

In [293]:
def StatenIslandCourseChildLevelBStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    StatenIslandQueueTeachersChildLevelB.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if StatenIslandTeachersChildLevelB.CurrentNumBusy < StatenIslandTeachersChildLevelB.NumberOfUnits and StatenIslandQueueTeachersChildLevelB.NumQueue() >= ClassSize:
        StatenIslandTeachersChildLevelB.Seize(1)
        for i in range(ClassSize):
            Student = StatenIslandQueueTeachersChildLevelB.Remove()
            StatenIslandTeachersChildLevelBWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "StatenIslandCourseChildLevelBStart", 0, Student)

In [294]:
def StatenIslandCourseChildLevelBStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "StatenIslandCourseChildLevelBEnd", LanguageCourseDurationLevelB, Student)

In [295]:
def StatenIslandCourseChildLevelBEnd(NextStudent):
    StatenIslandTeachersChildLevelB.Free(1)
    SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
    if StatenIslandTeachersChildLevelB.CurrentNumBusy < StatenIslandTeachersChildLevelB.NumberOfUnits and StatenIslandQueueTeachersChildLevelB.NumQueue() >= ClassSize:
        StatenIslandTeachersChildLevelB.Seize(1)
        for i in range(ClassSize):
            Student = StatenIslandQueueTeachersChildLevelB.Remove()
            StatenIslandTeachersChildLevelBWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "StatenIslandCourseChildLevelBStart", 0, Student)

In [296]:
def StatenIslandCourseChildLevelCStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    StatenIslandQueueTeachersChildLevelC.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if StatenIslandTeachersChildLevelC.CurrentNumBusy < StatenIslandTeachersChildLevelC.NumberOfUnits and StatenIslandQueueTeachersChildLevelC.NumQueue() >= ClassSize:
        StatenIslandTeachersChildLevelC.Seize(1)
        for i in range(ClassSize):
            Student = StatenIslandQueueTeachersChildLevelC.Remove()
            StatenIslandTeachersChildLevelCWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "StatenIslandCourseChildLevelCStart", 0, Student)

In [297]:
def StatenIslandCourseChildLevelCStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "StatenIslandCourseChildLevelCEnd", LanguageCourseDurationLevelC, Student)

In [298]:
def StatenIslandCourseChildLevelCEnd(NextStudent):
    StatenIslandTeachersChildLevelC.Free(1)
    SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
    if StatenIslandTeachersChildLevelC.CurrentNumBusy < StatenIslandTeachersChildLevelC.NumberOfUnits and StatenIslandQueueTeachersChildLevelC.NumQueue() >= ClassSize:
        StatenIslandTeachersChildLevelC.Seize(1)
        for i in range(ClassSize):
            Student = StatenIslandQueueTeachersChildLevelC.Remove()
            StatenIslandTeachersChildLevelCWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "StatenIslandCourseChildLevelCStart", 0, Student)

In [299]:
def StatenIslandCourseAdultLevelAStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    StatenIslandQueueTeachersAdultLevelA.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if StatenIslandTeachersAdultLevelA.CurrentNumBusy < StatenIslandTeachersAdultLevelA.NumberOfUnits and StatenIslandQueueTeachersAdultLevelA.NumQueue() >= ClassSize:
        StatenIslandTeachersAdultLevelA.Seize(1)
        for i in range(ClassSize):
            Student = StatenIslandQueueTeachersAdultLevelA.Remove()
            StatenIslandTeachersAdultLevelAWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "StatenIslandCourseAdultLevelAStart", 0, Student)

In [300]:
def StatenIslandCourseAdultLevelAStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "StatenIslandCourseAdultLevelAEnd", LanguageCourseDurationLevelA, Student)

In [301]:
def StatenIslandCourseAdultLevelAEnd(NextStudent):
    StatenIslandTeachersAdultLevelA.Free(1)
    if NextStudent.getEducationLevel() == "Less Than Bachelor":
        NextStudent.JoinCareerCoachingQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'StatenIslandCareerCoachingStartOfWaiting', 0, NextStudent)
    else:
        SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
        
    if StatenIslandTeachersAdultLevelA.CurrentNumBusy < StatenIslandTeachersAdultLevelA.NumberOfUnits and StatenIslandQueueTeachersAdultLevelA.NumQueue() >= ClassSize:
        StatenIslandTeachersAdultLevelA.Seize(1)
        for i in range(ClassSize):
            Student = StatenIslandQueueTeachersAdultLevelA.Remove()
            StatenIslandTeachersAdultLevelAWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "StatenIslandCourseAdultLevelAStart", 0, Student)

In [302]:
def StatenIslandCourseAdultLevelBStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    StatenIslandQueueTeachersAdultLevelB.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if StatenIslandTeachersAdultLevelB.CurrentNumBusy < StatenIslandTeachersAdultLevelB.NumberOfUnits and StatenIslandQueueTeachersAdultLevelB.NumQueue() >= ClassSize:
        StatenIslandTeachersAdultLevelB.Seize(1)
        for i in range(ClassSize):
            Student = StatenIslandQueueTeachersAdultLevelB.Remove()
            StatenIslandTeachersAdultLevelBWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "StatenIslandCourseAdultLevelBStart", 0, Student)

In [303]:
def StatenIslandCourseAdultLevelBStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "StatenIslandCourseAdultLevelBEnd", LanguageCourseDurationLevelB, Student)

In [304]:
def StatenIslandCourseAdultLevelBEnd(NextStudent):
    StatenIslandTeachersAdultLevelB.Free(1)
    if NextStudent.getEducationLevel() == "Less Than Bachelor":
        NextStudent.JoinCareerCoachingQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'StatenIslandCareerCoachingStartOfWaiting', 0, NextStudent)
    else:
        SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
        
    if StatenIslandTeachersAdultLevelB.CurrentNumBusy < StatenIslandTeachersAdultLevelB.NumberOfUnits and StatenIslandQueueTeachersAdultLevelB.NumQueue() >= ClassSize:
        StatenIslandTeachersAdultLevelB.Seize(1)
        for i in range(ClassSize):
            Student = StatenIslandQueueTeachersAdultLevelB.Remove()
            StatenIslandTeachersAdultLevelBWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "StatenIslandCourseAdultLevelBStart", 0, Student)

In [305]:
def StatenIslandCourseAdultLevelCStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    StatenIslandQueueTeachersAdultLevelC.Add(NewImmigrant)
    NewImmigrant.EnterLanguageCourseQueue()
    if StatenIslandTeachersAdultLevelC.CurrentNumBusy < StatenIslandTeachersAdultLevelC.NumberOfUnits and StatenIslandQueueTeachersAdultLevelC.NumQueue() >= ClassSize:
        StatenIslandTeachersAdultLevelC.Seize(1)
        for i in range(ClassSize):
            Student = StatenIslandQueueTeachersAdultLevelC.Remove()
            StatenIslandTeachersAdultLevelCWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "StatenIslandCourseAdultLevelCStart", 0, Student)

In [306]:
def StatenIslandCourseAdultLevelCStart(Student : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "StatenIslandCourseAdultLevelCEnd", LanguageCourseDurationLevelC, Student)

In [307]:
def StatenIslandCourseAdultLevelCEnd(NextStudent):
    StatenIslandTeachersAdultLevelC.Free(1)
    if NextStudent.getEducationLevel() == "Less Than Bachelor":
        NextStudent.JoinCareerCoachingQueueTime = SimClasses.Clock
        SimFunctions.SchedulePlus(Calendar, 'StatenIslandCareerCoachingStartOfWaiting', 0, NextStudent)
    else:
        SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NextStudent)
        
    if StatenIslandTeachersAdultLevelC.CurrentNumBusy < StatenIslandTeachersAdultLevelC.NumberOfUnits and StatenIslandQueueTeachersAdultLevelC.NumQueue() >= ClassSize:
        StatenIslandTeachersAdultLevelC.Seize(1)
        for i in range(ClassSize):
            Student = StatenIslandQueueTeachersAdultLevelC.Remove()
            StatenIslandTeachersAdultLevelCWaitTime.Record(SimClasses.Clock - Student.getJoinClassQueueTime())
            SimFunctions.SchedulePlus(Calendar, "StatenIslandCourseAdultLevelCStart", 0, Student)

In [308]:
def StatenIslandCareerCoachingStartOfWaiting(NewImmigrant : ImmigrantClass.Immigrant()):
    StatenIslandQueueCareerCoach.Add(NewImmigrant)
    NewImmigrant.EnterCareerCoachingQueue()
    if StatenIslandCareerCoach.CurrentNumBusy < StatenIslandCareerCoach.NumberOfUnits and StatenIslandQueueCareerCoach.NumQueue() > 0:
        StatenIslandCareerCoach.Seize(1)
        NextImmigrant = StatenIslandQueueCareerCoach.Remove()
        StatenIslandCareerCoachWaitTime.Record(SimClasses.Clock - NextImmigrant.getJoinCareerCoachingQueueTime())
        SimFunctions.SchedulePlus(Calendar, "StatenIslandCareerCoachingStart", 0, NextImmigrant)

In [309]:
def StatenIslandCareerCoachingStart(NewImmigrant : ImmigrantClass.Immigrant()):
    SimFunctions.SchedulePlus(Calendar, "StatenIslandCareerCoachingEnd", CareerCoachingSessionDuration, NewImmigrant)

In [310]:
def StatenIslandCareerCoachingEnd(NewImmigrant : ImmigrantClass.Immigrant()):
    StatenIslandCareerCoach.Free(1)
    SimFunctions.SchedulePlus(Calendar, "ProcessComplete", 0, NewImmigrant)
    if StatenIslandCareerCoach.CurrentNumBusy < StatenIslandCareerCoach.NumberOfUnits and StatenIslandQueueCareerCoach.NumQueue() > 0:
        StatenIslandCareerCoach.Seize(1)
        NextImmigrant = StatenIslandQueueCareerCoach.Remove()
        StatenIslandCareerCoachWaitTime.Record(SimClasses.Clock - NextImmigrant.getJoinCareerCoachingQueueTime())
        SimFunctions.SchedulePlus(Calendar, "StatenIslandCareerCoachingStart", 0, NextImmigrant)

In [311]:
def ProcessComplete(NewImmigrant : ImmigrantClass.Immigrant()):
    if NewImmigrant.getBorough() == "Brooklyn":
        BrooklynTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        if NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "Fluent":
            BrooklynFluentChildrenTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelA":
            BrooklynLevelAChildrenTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelB":
            BrooklynLevelBChildrenTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelC":
            BrooklynLevelCChildrenTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "Fluent" and NewImmigrant.getEducationLevel() == "Bachelor":
            BrooklynFluentEducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelA" and NewImmigrant.getEducationLevel() == "Bachelor":
            BrooklynLevelAEducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelB" and NewImmigrant.getEducationLevel() == "Bachelor":
            BrooklynLevelBEducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelC" and NewImmigrant.getEducationLevel() == "Bachelor":
            BrooklynLevelCEducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "Fluent" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
            BrooklynFluentUneducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelA" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
            BrooklynLevelAUneducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelB" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
            BrooklynLevelBUneducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelC" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
            BrooklynLevelCUneducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Old" and NewImmigrant.getEnglishProficiency() == "Fluent":
            BrooklynFluentOldTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Old" and NewImmigrant.getEnglishProficiency() == "LevelA":
            BrooklynLevelAOldTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Old" and NewImmigrant.getEnglishProficiency() == "LevelB":
            BrooklynLevelBOldTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Old" and NewImmigrant.getEnglishProficiency() == "LevelC":
            BrooklynLevelCOldTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        else:
            return
    
    elif NewImmigrant.getBorough() == "Manhattan":
        ManhattanTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        if NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "Fluent":
            ManhattanFluentChildrenTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelA":
            ManhattanLevelAChildrenTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelB":
            ManhattanLevelBChildrenTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelC":
            ManhattanLevelCChildrenTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "Fluent" and NewImmigrant.getEducationLevel() == "Bachelor":
            ManhattanFluentEducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelA" and NewImmigrant.getEducationLevel() == "Bachelor":
            ManhattanLevelAEducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelB" and NewImmigrant.getEducationLevel() == "Bachelor":
            ManhattanLevelBEducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelC" and NewImmigrant.getEducationLevel() == "Bachelor":
            ManhattanLevelCEducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "Fluent" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
            ManhattanFluentUneducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelA" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
            ManhattanLevelAUneducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelB" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
            ManhattanLevelBUneducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelC" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
            ManhattanLevelCUneducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Old" and NewImmigrant.getEnglishProficiency() == "Fluent":
            ManhattanFluentOldTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Old" and NewImmigrant.getEnglishProficiency() == "LevelA":
            ManhattanLevelAOldTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Old" and NewImmigrant.getEnglishProficiency() == "LevelB":
            ManhattanLevelBOldTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Old" and NewImmigrant.getEnglishProficiency() == "LevelC":
            ManhattanLevelCOldTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        else:
            return
    
    elif NewImmigrant.getBorough() == "Queens":
        QueensTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        if NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "Fluent":
            QueensFluentChildrenTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelA":
            QueensLevelAChildrenTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelB":
            QueensLevelBChildrenTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelC":
            QueensLevelCChildrenTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "Fluent" and NewImmigrant.getEducationLevel() == "Bachelor":
            QueensFluentEducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelA" and NewImmigrant.getEducationLevel() == "Bachelor":
            QueensLevelAEducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelB" and NewImmigrant.getEducationLevel() == "Bachelor":
            QueensLevelBEducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelC" and NewImmigrant.getEducationLevel() == "Bachelor":
            QueensLevelCEducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "Fluent" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
            QueensFluentUneducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelA" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
            QueensLevelAUneducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelB" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
            QueensLevelBUneducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelC" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
            QueensLevelCUneducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Old" and NewImmigrant.getEnglishProficiency() == "Fluent":
            QueensFluentOldTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Old" and NewImmigrant.getEnglishProficiency() == "LevelA":
            QueensLevelAOldTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Old" and NewImmigrant.getEnglishProficiency() == "LevelB":
            QueensLevelBOldTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Old" and NewImmigrant.getEnglishProficiency() == "LevelC":
            QueensLevelCOldTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        else:
            return
        
    elif NewImmigrant.getBorough() == "Bronx":
        BronxTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        if NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "Fluent":
            BronxFluentChildrenTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelA":
            BronxLevelAChildrenTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelB":
            BronxLevelBChildrenTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelC":
            BronxLevelCChildrenTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "Fluent" and NewImmigrant.getEducationLevel() == "Bachelor":
            BronxFluentEducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelA" and NewImmigrant.getEducationLevel() == "Bachelor":
            BronxLevelAEducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelB" and NewImmigrant.getEducationLevel() == "Bachelor":
            BronxLevelBEducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelC" and NewImmigrant.getEducationLevel() == "Bachelor":
            BronxLevelCEducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "Fluent" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
            BronxFluentUneducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelA" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
            BronxLevelAUneducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelB" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
            BronxLevelBUneducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelC" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
            BronxLevelCUneducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Old" and NewImmigrant.getEnglishProficiency() == "Fluent":
            BronxFluentOldTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Old" and NewImmigrant.getEnglishProficiency() == "LevelA":
            BronxLevelAOldTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Old" and NewImmigrant.getEnglishProficiency() == "LevelB":
            BronxLevelBOldTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Old" and NewImmigrant.getEnglishProficiency() == "LevelC":
            BronxLevelCOldTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        else:
            return
        
    elif NewImmigrant.getBorough() == "StatenIsland":
        StatenIslandTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        if NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "Fluent":
            StatenIslandFluentChildrenTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelA":
            StatenIslandLevelAChildrenTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelB":
            StatenIslandLevelBChildrenTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Child" and NewImmigrant.getEnglishProficiency() == "LevelC":
            StatenIslandLevelCChildrenTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "Fluent" and NewImmigrant.getEducationLevel() == "Bachelor":
            StatenIslandFluentEducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelA" and NewImmigrant.getEducationLevel() == "Bachelor":
            StatenIslandLevelAEducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelB" and NewImmigrant.getEducationLevel() == "Bachelor":
            StatenIslandLevelBEducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelC" and NewImmigrant.getEducationLevel() == "Bachelor":
            StatenIslandLevelCEducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "Fluent" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
            StatenIslandFluentUneducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelA" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
            StatenIslandLevelAUneducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelB" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
            StatenIslandLevelBUneducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Adult" and NewImmigrant.getEnglishProficiency() == "LevelC" and NewImmigrant.getEducationLevel() == "Less Than Bachelor":
            StatenIslandLevelCUneducatedAdultTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Old" and NewImmigrant.getEnglishProficiency() == "Fluent":
            StatenIslandFluentOldTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Old" and NewImmigrant.getEnglishProficiency() == "LevelA":
            StatenIslandLevelAOldTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Old" and NewImmigrant.getEnglishProficiency() == "LevelB":
            StatenIslandLevelBOldTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        elif NewImmigrant.getAge() == "Old" and NewImmigrant.getEnglishProficiency() == "LevelC":
            StatenIslandLevelCOldTotalProcessTime.Record(SimClasses.Clock - NewImmigrant.getTime())
        else:
            return
        
    else:
        print("ERROR : Unknown entity")
        return

In [348]:
#run simulation in batches of 5 because it keeps getting stuck for some reason
k = 6
BoroughDict = {"Brooklyn": 0, "Manhattan": 1, "Queens": 2, "Bronx": 3, "StatenIsland": 4}
AgeDict = {"Child": 0, "Adult": 1, "Old": 2}
Immigrants_successfully_processed_avg = np.zeros((5, 3))
pbar_outer = tqdm_notebook(total=Epochs, desc="Running Simulation")
for rep in range(Epochs):
    time.sleep(0.1)
    SimFunctions.SimFunctionsInit(Calendar)
    #Set resources 
    BrooklynLanguageAssessors.SetUnits(BrooklynNumberOfLanguageAssessors)
    BrooklynTeachersAdultLevelA.SetUnits(BrooklynNumberOfTeachersAdultLevelA)
    BrooklynTeachersAdultLevelB.SetUnits(BrooklynNumberOfTeachersAdultLevelB)
    BrooklynTeachersAdultLevelC.SetUnits(BrooklynNumberOfTeachersAdultLevelC)
    BrooklynTeachersChildLevelA.SetUnits(BrooklynNumberOfTeachersChildLevelA)
    BrooklynTeachersChildLevelB.SetUnits(BrooklynNumberOfTeachersChildLevelB)
    BrooklynTeachersChildLevelC.SetUnits(BrooklynNumberOfTeachersChildLevelC)
    BrooklynCareerCoach.SetUnits(BrooklynNumberOfCareerCoaches)
    
    ManhattanLanguageAssessors.SetUnits(ManhattanNumberOfLanguageAssessors)
    ManhattanTeachersAdultLevelA.SetUnits(ManhattanNumberOfTeachersAdultLevelA)
    ManhattanTeachersAdultLevelB.SetUnits(ManhattanNumberOfTeachersAdultLevelB)
    ManhattanTeachersAdultLevelC.SetUnits(ManhattanNumberOfTeachersAdultLevelC)
    ManhattanTeachersChildLevelA.SetUnits(ManhattanNumberOfTeachersChildLevelA)
    ManhattanTeachersChildLevelB.SetUnits(ManhattanNumberOfTeachersChildLevelB)
    ManhattanTeachersChildLevelC.SetUnits(ManhattanNumberOfTeachersChildLevelC)
    ManhattanCareerCoach.SetUnits(ManhattanNumberOfCareerCoaches)
    
    QueensLanguageAssessors.SetUnits(QueensNumberOfLanguageAssessors)
    QueensTeachersAdultLevelA.SetUnits(QueensNumberOfTeachersAdultLevelA)
    QueensTeachersAdultLevelB.SetUnits(QueensNumberOfTeachersAdultLevelB)
    QueensTeachersAdultLevelC.SetUnits(QueensNumberOfTeachersAdultLevelC)
    QueensTeachersChildLevelA.SetUnits(QueensNumberOfTeachersChildLevelA)
    QueensTeachersChildLevelB.SetUnits(QueensNumberOfTeachersChildLevelB)
    QueensTeachersChildLevelC.SetUnits(QueensNumberOfTeachersChildLevelC)
    QueensCareerCoach.SetUnits(QueensNumberOfCareerCoaches)
    
    BronxLanguageAssessors.SetUnits(BronxNumberOfLanguageAssessors)
    BronxTeachersAdultLevelA.SetUnits(BronxNumberOfTeachersAdultLevelA)
    BronxTeachersAdultLevelB.SetUnits(BronxNumberOfTeachersAdultLevelB)
    BronxTeachersAdultLevelC.SetUnits(BronxNumberOfTeachersAdultLevelC)
    BronxTeachersChildLevelA.SetUnits(BronxNumberOfTeachersChildLevelA)
    BronxTeachersChildLevelB.SetUnits(BronxNumberOfTeachersChildLevelB)
    BronxTeachersChildLevelC.SetUnits(BronxNumberOfTeachersChildLevelC)
    BronxCareerCoach.SetUnits(BronxNumberOfCareerCoaches)
    
    StatenIslandLanguageAssessors.SetUnits(StatenIslandNumberOfLanguageAssessors)
    StatenIslandTeachersAdultLevelA.SetUnits(StatenIslandNumberOfTeachersAdultLevelA)
    StatenIslandTeachersAdultLevelB.SetUnits(StatenIslandNumberOfTeachersAdultLevelB)
    StatenIslandTeachersAdultLevelC.SetUnits(StatenIslandNumberOfTeachersAdultLevelC)
    StatenIslandTeachersChildLevelA.SetUnits(StatenIslandNumberOfTeachersChildLevelA)
    StatenIslandTeachersChildLevelB.SetUnits(StatenIslandNumberOfTeachersChildLevelB)
    StatenIslandTeachersChildLevelC.SetUnits(StatenIslandNumberOfTeachersChildLevelC)
    StatenIslandCareerCoach.SetUnits(StatenIslandNumberOfCareerCoaches)
    
    SimFunctions.Schedule(Calendar, "Arrival", SimRNG.Expon(ArrivalRate2022, 5))
    SimFunctions.Schedule(Calendar, "EndSimulation", RunLength)
    
    Immigrants_processed_by_borough_and_age = np.zeros((5, 3))
    #Main simulation loop
    pbar_inner = tqdm_notebook(total=Calendar.N(), desc=f"Iteration {rep + 1} in process", leave=False)
    while Calendar.N() > 0:
        ''''''
        NextEvent = Calendar.Remove()
        SimClasses.Clock = NextEvent.EventTime

        if NextEvent.EventType == 'Arrival':
            Arrival()
        
        ##BROOKLYN##
        elif NextEvent.EventType == 'BrooklynVerbalEvaluationStartOfWaiting':
            BrooklynLanguageAssessorsQueueLength.append(BrooklynQueueLanguageAssessors.NumQueue())
            BrooklynVerbalEvaluationStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynVerbalEvaluationStart':
            BrooklynVerbalEvaluationStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynVerbalEvaluationEnd':
            BrooklynVerbalEvaluationEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynCourseChildLevelAStartOfWaiting':
            BrooklynTeachersChildLevelAQueueLength.append(BrooklynQueueTeachersChildLevelA.NumQueue())
            BrooklynCourseChildLevelAStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynCourseChildLevelAStart':
            BrooklynCourseChildLevelAStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynCourseChildLevelAEnd':
            BrooklynCourseChildLevelAEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynCourseChildLevelBStartOfWaiting':
            BrooklynTeachersChildLevelBQueueLength.append(BrooklynQueueTeachersChildLevelB.NumQueue())
            BrooklynCourseChildLevelBStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynCourseChildLevelBStart':
            BrooklynCourseChildLevelBStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynCourseChildLevelBEnd':
            BrooklynCourseChildLevelBEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynCourseChildLevelCStartOfWaiting':
            BrooklynTeachersChildLevelCQueueLength.append(BrooklynQueueTeachersChildLevelC.NumQueue())
            BrooklynCourseChildLevelCStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynCourseChildLevelCStart':
            BrooklynCourseChildLevelCStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynCourseChildLevelCEnd':
            BrooklynCourseChildLevelCEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynCourseAdultLevelAStartOfWaiting':
            BrooklynTeachersAdultLevelAQueueLength.append(BrooklynQueueTeachersAdultLevelA.NumQueue())
            BrooklynCourseAdultLevelAStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynCourseAdultLevelAStart':
            BrooklynCourseAdultLevelAStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynCourseAdultLevelAEnd':
            BrooklynCourseAdultLevelAEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynCourseAdultLevelBStartOfWaiting':
            BrooklynTeachersAdultLevelBQueueLength.append(BrooklynQueueTeachersAdultLevelB.NumQueue())
            BrooklynCourseAdultLevelBStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynCourseAdultLevelBStart':
            BrooklynCourseAdultLevelBStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynCourseAdultLevelBEnd':
            BrooklynCourseAdultLevelBEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynCourseAdultLevelCStartOfWaiting':
            BrooklynTeachersAdultLevelCQueueLength.append(BrooklynQueueTeachersAdultLevelC.NumQueue())
            BrooklynCourseAdultLevelCStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynCourseAdultLevelCStart':
            BrooklynCourseAdultLevelCStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynCourseAdultLevelCEnd':
            BrooklynCourseAdultLevelCEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynCareerCoachingStartOfWaiting':
            BrooklynCareerCoachesQueueLength.append(BrooklynQueueCareerCoach.NumQueue())
            BrooklynCareerCoachingStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynCareerCoachingStart':
            BrooklynCareerCoachingStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BrooklynCareerCoachingEnd':
            BrooklynCareerCoachingEnd(NextEvent.WhichObject)
        #############
        ##MANHATTAN##
        elif NextEvent.EventType == 'ManhattanVerbalEvaluationStartOfWaiting':
            ManhattanLanguageAssessorsQueueLength.append(ManhattanQueueLanguageAssessors.NumQueue())
            ManhattanVerbalEvaluationStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanVerbalEvaluationStart':
            ManhattanVerbalEvaluationStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanVerbalEvaluationEnd':
            ManhattanVerbalEvaluationEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanCourseChildLevelAStartOfWaiting':
            ManhattanTeachersChildLevelAQueueLength.append(ManhattanQueueTeachersChildLevelA.NumQueue())
            ManhattanCourseChildLevelAStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanCourseChildLevelAStart':
            ManhattanCourseChildLevelAStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanCourseChildLevelAEnd':
            ManhattanCourseChildLevelAEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanCourseChildLevelBStartOfWaiting':
            ManhattanTeachersChildLevelBQueueLength.append(ManhattanQueueTeachersChildLevelB.NumQueue())
            ManhattanCourseChildLevelBStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanCourseChildLevelBStart':
            ManhattanCourseChildLevelBStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanCourseChildLevelBEnd':
            ManhattanCourseChildLevelBEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanCourseChildLevelCStartOfWaiting':
            ManhattanTeachersChildLevelCQueueLength.append(ManhattanQueueTeachersChildLevelC.NumQueue())
            ManhattanCourseChildLevelCStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanCourseChildLevelCStart':
            ManhattanCourseChildLevelCStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanCourseChildLevelCEnd':
            ManhattanCourseChildLevelCEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanCourseAdultLevelAStartOfWaiting':
            ManhattanTeachersAdultLevelAQueueLength.append(ManhattanQueueTeachersAdultLevelA.NumQueue())
            ManhattanCourseAdultLevelAStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanCourseAdultLevelAStart':
            ManhattanCourseAdultLevelAStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanCourseAdultLevelAEnd':
            ManhattanCourseAdultLevelAEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanCourseAdultLevelBStartOfWaiting':
            ManhattanTeachersAdultLevelBQueueLength.append(ManhattanQueueTeachersAdultLevelB.NumQueue())
            ManhattanCourseAdultLevelBStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanCourseAdultLevelBStart':
            ManhattanCourseAdultLevelBStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanCourseAdultLevelBEnd':
            ManhattanCourseAdultLevelBEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanCourseAdultLevelCStartOfWaiting':
            ManhattanTeachersAdultLevelCQueueLength.append(ManhattanQueueTeachersAdultLevelC.NumQueue())
            ManhattanCourseAdultLevelCStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanCourseAdultLevelCStart':
            ManhattanCourseAdultLevelCStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanCourseAdultLevelCEnd':
            ManhattanCourseAdultLevelCEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanCareerCoachingStartOfWaiting':
            ManhattanCareerCoachesQueueLength.append(ManhattanQueueCareerCoach.NumQueue())
            ManhattanCareerCoachingStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanCareerCoachingStart':
            ManhattanCareerCoachingStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'ManhattanCareerCoachingEnd':
            ManhattanCareerCoachingEnd(NextEvent.WhichObject)
        #############
        ##QUEENS##
        elif NextEvent.EventType == 'QueensVerbalEvaluationStartOfWaiting':
            QueensLanguageAssessorsQueueLength.append(QueensQueueLanguageAssessors.NumQueue())
            QueensVerbalEvaluationStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensVerbalEvaluationStart':
            QueensVerbalEvaluationStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensVerbalEvaluationEnd':
            QueensVerbalEvaluationEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensCourseChildLevelAStartOfWaiting':
            QueensTeachersChildLevelAQueueLength.append(QueensQueueTeachersChildLevelA.NumQueue())
            QueensCourseChildLevelAStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensCourseChildLevelAStart':
            QueensCourseChildLevelAStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensCourseChildLevelAEnd':
            QueensCourseChildLevelAEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensCourseChildLevelBStartOfWaiting':
            QueensTeachersChildLevelBQueueLength.append(QueensQueueTeachersChildLevelB.NumQueue())
            QueensCourseChildLevelBStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensCourseChildLevelBStart':
            QueensCourseChildLevelBStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensCourseChildLevelBEnd':
            QueensCourseChildLevelBEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensCourseChildLevelCStartOfWaiting':
            QueensTeachersChildLevelCQueueLength.append(QueensQueueTeachersChildLevelC.NumQueue())
            QueensCourseChildLevelCStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensCourseChildLevelCStart':
            QueensCourseChildLevelCStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensCourseChildLevelCEnd':
            QueensCourseChildLevelCEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensCourseAdultLevelAStartOfWaiting':
            QueensTeachersAdultLevelAQueueLength.append(QueensQueueTeachersAdultLevelA.NumQueue())
            QueensCourseAdultLevelAStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensCourseAdultLevelAStart':
            QueensCourseAdultLevelAStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensCourseAdultLevelAEnd':
            QueensCourseAdultLevelAEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensCourseAdultLevelBStartOfWaiting':
            QueensTeachersAdultLevelBQueueLength.append(QueensQueueTeachersAdultLevelB.NumQueue())
            QueensCourseAdultLevelBStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensCourseAdultLevelBStart':
            QueensCourseAdultLevelBStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensCourseAdultLevelBEnd':
            QueensCourseAdultLevelBEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensCourseAdultLevelCStartOfWaiting':
            QueensTeachersAdultLevelCQueueLength.append(QueensQueueTeachersAdultLevelC.NumQueue())
            QueensCourseAdultLevelCStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensCourseAdultLevelCStart':
            QueensCourseAdultLevelCStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensCourseAdultLevelCEnd':
            QueensCourseAdultLevelCEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensCareerCoachingStartOfWaiting':
            QueensCareerCoachesQueueLength.append(QueensQueueCareerCoach.NumQueue())
            QueensCareerCoachingStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensCareerCoachingStart':
            QueensCareerCoachingStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'QueensCareerCoachingEnd':
            QueensCareerCoachingEnd(NextEvent.WhichObject)
        ##########
        ##BRONX##
        elif NextEvent.EventType == 'BronxVerbalEvaluationStartOfWaiting':
            BronxLanguageAssessorsQueueLength.append(BronxQueueLanguageAssessors.NumQueue())
            BronxVerbalEvaluationStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxVerbalEvaluationStart':
            BronxVerbalEvaluationStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxVerbalEvaluationEnd':
            BronxVerbalEvaluationEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxCourseChildLevelAStartOfWaiting':
            BronxTeachersChildLevelAQueueLength.append(BronxQueueTeachersChildLevelA.NumQueue())
            BronxCourseChildLevelAStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxCourseChildLevelAStart':
            BronxCourseChildLevelAStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxCourseChildLevelAEnd':
            BronxCourseChildLevelAEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxCourseChildLevelBStartOfWaiting':
            BronxTeachersChildLevelBQueueLength.append(BronxQueueTeachersChildLevelB.NumQueue())
            BronxCourseChildLevelBStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxCourseChildLevelBStart':
            BronxCourseChildLevelBStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxCourseChildLevelBEnd':
            BronxCourseChildLevelBEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxCourseChildLevelCStartOfWaiting':
            BronxTeachersChildLevelCQueueLength.append(BronxQueueTeachersChildLevelC.NumQueue())
            BronxCourseChildLevelCStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxCourseChildLevelCStart':
            BronxCourseChildLevelCStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxCourseChildLevelCEnd':
            BronxCourseChildLevelCEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxCourseAdultLevelAStartOfWaiting':
            BronxTeachersAdultLevelAQueueLength.append(BronxQueueTeachersAdultLevelA.NumQueue())
            BronxCourseAdultLevelAStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxCourseAdultLevelAStart':
            BronxCourseAdultLevelAStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxCourseAdultLevelAEnd':
            BronxCourseAdultLevelAEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxCourseAdultLevelBStartOfWaiting':
            BronxTeachersAdultLevelBQueueLength.append(BronxQueueTeachersAdultLevelB.NumQueue())
            BronxCourseAdultLevelBStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxCourseAdultLevelBStart':
            BronxCourseAdultLevelBStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxCourseAdultLevelBEnd':
            BronxCourseAdultLevelBEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxCourseAdultLevelCStartOfWaiting':
            BronxTeachersAdultLevelCQueueLength.append(BronxQueueTeachersAdultLevelC.NumQueue())
            BronxCourseAdultLevelCStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxCourseAdultLevelCStart':
            BronxCourseAdultLevelCStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxCourseAdultLevelCEnd':
            BronxCourseAdultLevelCEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxCareerCoachingStartOfWaiting':
            BronxCareerCoachesQueueLength.append(BronxQueueCareerCoach.NumQueue())
            BronxCareerCoachingStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxCareerCoachingStart':
            BronxCareerCoachingStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'BronxCareerCoachingEnd':
            BronxCareerCoachingEnd(NextEvent.WhichObject)
        #########
        ##STATENISLAND##
        elif NextEvent.EventType == 'StatenIslandVerbalEvaluationStartOfWaiting':
            StatenIslandLanguageAssessorsQueueLength.append(StatenIslandQueueLanguageAssessors.NumQueue())
            StatenIslandVerbalEvaluationStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandVerbalEvaluationStart':
            StatenIslandVerbalEvaluationStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandVerbalEvaluationEnd':
            StatenIslandVerbalEvaluationEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandCourseChildLevelAStartOfWaiting':
            StatenIslandTeachersChildLevelAQueueLength.append(StatenIslandQueueTeachersChildLevelA.NumQueue())
            StatenIslandCourseChildLevelAStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandCourseChildLevelAStart':
            StatenIslandCourseChildLevelAStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandCourseChildLevelAEnd':
            StatenIslandCourseChildLevelAEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandCourseChildLevelBStartOfWaiting':
            StatenIslandTeachersChildLevelBQueueLength.append(StatenIslandQueueTeachersChildLevelB.NumQueue())
            StatenIslandCourseChildLevelBStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandCourseChildLevelBStart':
            StatenIslandCourseChildLevelBStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandCourseChildLevelBEnd':
            StatenIslandCourseChildLevelBEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandCourseChildLevelCStartOfWaiting':
            StatenIslandTeachersChildLevelCQueueLength.append(StatenIslandQueueTeachersChildLevelC.NumQueue())
            StatenIslandCourseChildLevelCStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandCourseChildLevelCStart':
            StatenIslandCourseChildLevelCStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandCourseChildLevelCEnd':
            StatenIslandCourseChildLevelCEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandCourseAdultLevelAStartOfWaiting':
            StatenIslandTeachersAdultLevelAQueueLength.append(StatenIslandQueueTeachersAdultLevelA.NumQueue())
            StatenIslandCourseAdultLevelAStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandCourseAdultLevelAStart':
            StatenIslandCourseAdultLevelAStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandCourseAdultLevelAEnd':
            StatenIslandCourseAdultLevelAEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandCourseAdultLevelBStartOfWaiting':
            StatenIslandTeachersAdultLevelBQueueLength.append(StatenIslandQueueTeachersAdultLevelB.NumQueue())
            StatenIslandCourseAdultLevelBStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandCourseAdultLevelBStart':
            StatenIslandCourseAdultLevelBStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandCourseAdultLevelBEnd':
            StatenIslandCourseAdultLevelBEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandCourseAdultLevelCStartOfWaiting':
            StatenIslandTeachersAdultLevelCQueueLength.append(StatenIslandQueueTeachersAdultLevelC.NumQueue())
            StatenIslandCourseAdultLevelCStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandCourseAdultLevelCStart':
            StatenIslandCourseAdultLevelCStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandCourseAdultLevelCEnd':
            StatenIslandCourseAdultLevelCEnd(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandCareerCoachingStartOfWaiting':
            StatenIslandCareerCoachesQueueLength.append(StatenIslandQueueCareerCoach.NumQueue())
            StatenIslandCareerCoachingStartOfWaiting(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandCareerCoachingStart':
            StatenIslandCareerCoachingStart(NextEvent.WhichObject)
            
        elif NextEvent.EventType == 'StatenIslandCareerCoachingEnd':
            StatenIslandCareerCoachingEnd(NextEvent.WhichObject)
        #################
        elif NextEvent.EventType == 'ProcessComplete':
            ProcessComplete(NextEvent.WhichObject)
            Immigrants_processed_by_borough_and_age[BoroughDict[NextEvent.WhichObject.getBorough()], AgeDict[NextEvent.WhichObject.getAge()]] += 1
        elif NextEvent.EventType == 'EndSimulation':
            break
        else:
            print(NextEvent.EventType)
            print("ERROR : Unknown event type")
            break
        
        pbar_inner.update(1)
    pbar_inner.close()

    # At end of replication, store stats in lists
    BrooklynLanguageAssessorsWaitTimeAVG.append(BrooklynLanguageAssessorsWaitTime.Mean())
    BrooklynLanguageAssessorsQueueLengthAVG.append(BrooklynQueueLanguageAssessors.Mean())
    BrooklynLanguageAssessorsBusyAVG.append(BrooklynLanguageAssessors.Mean())
    
    
    BrooklynTeachersAdultLevelAWaitTimeAVG.append(BrooklynTeachersAdultLevelAWaitTime.Mean())
    BrooklynTeachersAdultLevelAQueueLengthAVG.append(BrooklynQueueTeachersAdultLevelA.Mean())
    BrooklynTeachersAdultLevelABusyAVG.append(BrooklynTeachersAdultLevelA.Mean())
    
    BrooklynTeachersAdultLevelBWaitTimeAVG.append(BrooklynTeachersAdultLevelBWaitTime.Mean())
    BrooklynTeachersAdultLevelBQueueLengthAVG.append(BrooklynQueueTeachersAdultLevelB.Mean())
    BrooklynTeachersAdultLevelBBusyAVG.append(BrooklynTeachersAdultLevelB.Mean())
    
    BrooklynTeachersAdultLevelCWaitTimeAVG.append(BrooklynTeachersAdultLevelCWaitTime.Mean())
    BrooklynTeachersAdultLevelCQueueLengthAVG.append(BrooklynQueueTeachersAdultLevelC.Mean())
    BrooklynTeachersAdultLevelCBusyAVG.append(BrooklynTeachersAdultLevelC.Mean())
    
    BrooklynTeachersChildLevelAWaitTimeAVG.append(BrooklynTeachersChildLevelAWaitTime.Mean())
    BrooklynTeachersChildLevelAQueueLengthAVG.append(BrooklynQueueTeachersChildLevelA.Mean())
    BrooklynTeachersChildLevelABusyAVG.append(BrooklynTeachersChildLevelA.Mean())
    
    BrooklynTeachersChildLevelBWaitTimeAVG.append(BrooklynTeachersChildLevelBWaitTime.Mean())
    BrooklynTeachersChildLevelBQueueLengthAVG.append(BrooklynQueueTeachersChildLevelB.Mean())
    BrooklynTeachersChildLevelBBusyAVG.append(BrooklynTeachersChildLevelB.Mean())
    
    BrooklynTeachersChildLevelCWaitTimeAVG.append(BrooklynTeachersChildLevelCWaitTime.Mean())
    BrooklynTeachersChildLevelCQueueLengthAVG.append(BrooklynQueueTeachersChildLevelC.Mean())
    BrooklynTeachersChildLevelCBusyAVG.append(BrooklynTeachersChildLevelC.Mean())
    
    BrooklynCareerCoachWaitTimeAVG.append(BrooklynCareerCoachWaitTime.Mean())
    BrooklynCareerCoachQueueLengthAVG.append(BrooklynQueueCareerCoach.Mean())
    BrooklynCareerCoachBusyAVG.append(BrooklynCareerCoach.Mean())
    
    BrooklynFluentChildrenTotalProcessTimeAVG.append(BrooklynFluentChildrenTotalProcessTime.Mean())
    BrooklynLevelAChildrenTotalProcessTimeAVG.append(BrooklynLevelAChildrenTotalProcessTime.Mean())
    BrooklynLevelBChildrenTotalProcessTimeAVG.append(BrooklynLevelBChildrenTotalProcessTime.Mean())
    BrooklynLevelCChildrenTotalProcessTimeAVG.append(BrooklynLevelCChildrenTotalProcessTime.Mean())
    
    BrooklynFluentEducatedAdultTotalProcessTimeAVG.append(BrooklynFluentEducatedAdultTotalProcessTime.Mean())
    BrooklynLevelAEducatedAdultTotalProcessTimeAVG.append(BrooklynLevelAEducatedAdultTotalProcessTime.Mean())
    BrooklynLevelBEducatedAdultTotalProcessTimeAVG.append(BrooklynLevelBEducatedAdultTotalProcessTime.Mean())
    BrooklynLevelCEducatedAdultTotalProcessTimeAVG.append(BrooklynLevelCEducatedAdultTotalProcessTime.Mean())
    BrooklynFluentUneducatedAdultTotalProcessTimeAVG.append(BrooklynFluentUneducatedAdultTotalProcessTime.Mean())
    BrooklynLevelAUneducatedAdultTotalProcessTimeAVG.append(BrooklynLevelAUneducatedAdultTotalProcessTime.Mean())
    BrooklynLevelBUneducatedAdultTotalProcessTimeAVG.append(BrooklynLevelBUneducatedAdultTotalProcessTime.Mean())
    BrooklynLevelCUneducatedAdultTotalProcessTimeAVG.append(BrooklynLevelCUneducatedAdultTotalProcessTime.Mean())
    
    BrooklynFluentOldTotalProcessTimeAVG.append(BrooklynFluentOldTotalProcessTime.Mean())
    BrooklynLevelAOldTotalProcessTimeAVG.append(BrooklynLevelAOldTotalProcessTime.Mean())
    BrooklynLevelBOldTotalProcessTimeAVG.append(BrooklynLevelBOldTotalProcessTime.Mean())
    BrooklynLevelCOldTotalProcessTimeAVG.append(BrooklynLevelCOldTotalProcessTime.Mean())
    
    BrooklynTotalProcessTimeAVG.append(BrooklynTotalProcessTime.Mean())
    
    ManhattanLanguageAssessorsWaitTimeAVG.append(ManhattanLanguageAssessorsWaitTime.Mean())
    ManhattanLanguageAssessorsQueueLengthAVG.append(ManhattanQueueLanguageAssessors.Mean())
    ManhattanLanguageAssessorsBusyAVG.append(ManhattanLanguageAssessors.Mean())
    
    ManhattanTeachersAdultLevelAWaitTimeAVG.append(ManhattanTeachersAdultLevelAWaitTime.Mean())
    ManhattanTeachersAdultLevelAQueueLengthAVG.append(ManhattanQueueTeachersAdultLevelA.Mean())
    ManhattanTeachersAdultLevelABusyAVG.append(ManhattanTeachersAdultLevelA.Mean())
    
    ManhattanTeachersAdultLevelBWaitTimeAVG.append(ManhattanTeachersAdultLevelBWaitTime.Mean())
    ManhattanTeachersAdultLevelBQueueLengthAVG.append(ManhattanQueueTeachersAdultLevelB.Mean())
    ManhattanTeachersAdultLevelBBusyAVG.append(ManhattanTeachersAdultLevelB.Mean())
    
    ManhattanTeachersAdultLevelCWaitTimeAVG.append(ManhattanTeachersAdultLevelCWaitTime.Mean())
    ManhattanTeachersAdultLevelCQueueLengthAVG.append(ManhattanQueueTeachersAdultLevelC.Mean())
    ManhattanTeachersAdultLevelCBusyAVG.append(ManhattanTeachersAdultLevelC.Mean())
    
    ManhattanTeachersChildLevelAWaitTimeAVG.append(ManhattanTeachersChildLevelAWaitTime.Mean())
    ManhattanTeachersChildLevelAQueueLengthAVG.append(ManhattanQueueTeachersChildLevelA.Mean())
    ManhattanTeachersChildLevelABusyAVG.append(ManhattanTeachersChildLevelA.Mean())
    
    ManhattanTeachersChildLevelBWaitTimeAVG.append(ManhattanTeachersChildLevelBWaitTime.Mean())
    ManhattanTeachersChildLevelBQueueLengthAVG.append(ManhattanQueueTeachersChildLevelB.Mean())
    ManhattanTeachersChildLevelBBusyAVG.append(ManhattanTeachersChildLevelB.Mean())
    
    ManhattanTeachersChildLevelCWaitTimeAVG.append(ManhattanTeachersChildLevelCWaitTime.Mean())
    ManhattanTeachersChildLevelCQueueLengthAVG.append(ManhattanQueueTeachersChildLevelC.Mean())
    ManhattanTeachersChildLevelCBusyAVG.append(ManhattanTeachersChildLevelC.Mean())
    
    ManhattanCareerCoachWaitTimeAVG.append(ManhattanCareerCoachWaitTime.Mean())
    ManhattanCareerCoachQueueLengthAVG.append(ManhattanQueueCareerCoach.Mean())
    ManhattanCareerCoachBusyAVG.append(ManhattanCareerCoach.Mean())
    
    ManhattanFluentChildrenTotalProcessTimeAVG.append(ManhattanFluentChildrenTotalProcessTime.Mean())
    ManhattanLevelAChildrenTotalProcessTimeAVG.append(ManhattanLevelAChildrenTotalProcessTime.Mean())
    ManhattanLevelBChildrenTotalProcessTimeAVG.append(ManhattanLevelBChildrenTotalProcessTime.Mean())
    ManhattanLevelCChildrenTotalProcessTimeAVG.append(ManhattanLevelCChildrenTotalProcessTime.Mean())
    
    ManhattanFluentEducatedAdultTotalProcessTimeAVG.append(ManhattanFluentEducatedAdultTotalProcessTime.Mean())
    ManhattanLevelAEducatedAdultTotalProcessTimeAVG.append(ManhattanLevelAEducatedAdultTotalProcessTime.Mean())
    ManhattanLevelBEducatedAdultTotalProcessTimeAVG.append(ManhattanLevelBEducatedAdultTotalProcessTime.Mean())
    ManhattanLevelCEducatedAdultTotalProcessTimeAVG.append(ManhattanLevelCEducatedAdultTotalProcessTime.Mean())
    ManhattanFluentUneducatedAdultTotalProcessTimeAVG.append(ManhattanFluentUneducatedAdultTotalProcessTime.Mean())
    ManhattanLevelAUneducatedAdultTotalProcessTimeAVG.append(ManhattanLevelAUneducatedAdultTotalProcessTime.Mean())
    ManhattanLevelBUneducatedAdultTotalProcessTimeAVG.append(ManhattanLevelBUneducatedAdultTotalProcessTime.Mean())
    ManhattanLevelCUneducatedAdultTotalProcessTimeAVG.append(ManhattanLevelCUneducatedAdultTotalProcessTime.Mean())
    
    ManhattanFluentOldTotalProcessTimeAVG.append(ManhattanFluentOldTotalProcessTime.Mean())
    ManhattanLevelAOldTotalProcessTimeAVG.append(ManhattanLevelAOldTotalProcessTime.Mean())
    ManhattanLevelBOldTotalProcessTimeAVG.append(ManhattanLevelBOldTotalProcessTime.Mean())
    ManhattanLevelCOldTotalProcessTimeAVG.append(ManhattanLevelCOldTotalProcessTime.Mean())
    
    ManhattanTotalProcessTimeAVG.append(ManhattanTotalProcessTime.Mean())
    
    QueensLanguageAssessorsWaitTimeAVG.append(QueensLanguageAssessorsWaitTime.Mean())
    QueensLanguageAssessorsQueueLengthAVG.append(QueensQueueLanguageAssessors.Mean())
    QueensLanguageAssessorsBusyAVG.append(QueensLanguageAssessors.Mean())
    
    QueensTeachersAdultLevelAWaitTimeAVG.append(QueensTeachersAdultLevelAWaitTime.Mean())
    QueensTeachersAdultLevelAQueueLengthAVG.append(QueensQueueTeachersAdultLevelA.Mean())
    QueensTeachersAdultLevelABusyAVG.append(QueensTeachersAdultLevelA.Mean())
    
    QueensTeachersAdultLevelBWaitTimeAVG.append(QueensTeachersAdultLevelBWaitTime.Mean())
    QueensTeachersAdultLevelBQueueLengthAVG.append(QueensQueueTeachersAdultLevelB.Mean())
    QueensTeachersAdultLevelBBusyAVG.append(QueensTeachersAdultLevelB.Mean())
    
    QueensTeachersAdultLevelCWaitTimeAVG.append(QueensTeachersAdultLevelCWaitTime.Mean())
    QueensTeachersAdultLevelCQueueLengthAVG.append(QueensQueueTeachersAdultLevelC.Mean())
    QueensTeachersAdultLevelCBusyAVG.append(QueensTeachersAdultLevelC.Mean())
    
    QueensTeachersChildLevelAWaitTimeAVG.append(QueensTeachersChildLevelAWaitTime.Mean())
    QueensTeachersChildLevelAQueueLengthAVG.append(QueensQueueTeachersChildLevelA.Mean())
    QueensTeachersChildLevelABusyAVG.append(QueensTeachersChildLevelA.Mean())
    
    QueensTeachersChildLevelBWaitTimeAVG.append(QueensTeachersChildLevelBWaitTime.Mean())
    QueensTeachersChildLevelBQueueLengthAVG.append(QueensQueueTeachersChildLevelB.Mean())
    QueensTeachersChildLevelBBusyAVG.append(QueensTeachersChildLevelB.Mean())
    
    QueensTeachersChildLevelCWaitTimeAVG.append(QueensTeachersChildLevelCWaitTime.Mean())
    QueensTeachersChildLevelCQueueLengthAVG.append(QueensQueueTeachersChildLevelC.Mean())
    QueensTeachersChildLevelCBusyAVG.append(QueensTeachersChildLevelC.Mean())
    
    QueensCareerCoachWaitTimeAVG.append(QueensCareerCoachWaitTime.Mean())
    QueensCareerCoachQueueLengthAVG.append(QueensQueueCareerCoach.Mean())
    QueensCareerCoachBusyAVG.append(QueensCareerCoach.Mean())
    
    QueensFluentChildrenTotalProcessTimeAVG.append(QueensFluentChildrenTotalProcessTime.Mean())
    QueensLevelAChildrenTotalProcessTimeAVG.append(QueensLevelAChildrenTotalProcessTime.Mean())
    QueensLevelBChildrenTotalProcessTimeAVG.append(QueensLevelBChildrenTotalProcessTime.Mean())
    QueensLevelCChildrenTotalProcessTimeAVG.append(QueensLevelCChildrenTotalProcessTime.Mean())
    
    QueensFluentEducatedAdultTotalProcessTimeAVG.append(QueensFluentEducatedAdultTotalProcessTime.Mean())
    QueensLevelAEducatedAdultTotalProcessTimeAVG.append(QueensLevelAEducatedAdultTotalProcessTime.Mean())
    QueensLevelBEducatedAdultTotalProcessTimeAVG.append(QueensLevelBEducatedAdultTotalProcessTime.Mean())
    QueensLevelCEducatedAdultTotalProcessTimeAVG.append(QueensLevelCEducatedAdultTotalProcessTime.Mean())
    QueensFluentUneducatedAdultTotalProcessTimeAVG.append(QueensFluentUneducatedAdultTotalProcessTime.Mean())
    QueensLevelAUneducatedAdultTotalProcessTimeAVG.append(QueensLevelAUneducatedAdultTotalProcessTime.Mean())
    QueensLevelBUneducatedAdultTotalProcessTimeAVG.append(QueensLevelBUneducatedAdultTotalProcessTime.Mean())
    QueensLevelCUneducatedAdultTotalProcessTimeAVG.append(QueensLevelCUneducatedAdultTotalProcessTime.Mean())
    
    QueensFluentOldTotalProcessTimeAVG.append(QueensFluentOldTotalProcessTime.Mean())
    QueensLevelAOldTotalProcessTimeAVG.append(QueensLevelAOldTotalProcessTime.Mean())
    QueensLevelBOldTotalProcessTimeAVG.append(QueensLevelBOldTotalProcessTime.Mean())
    QueensLevelCOldTotalProcessTimeAVG.append(QueensLevelCOldTotalProcessTime.Mean())
    
    QueensTotalProcessTimeAVG.append(QueensTotalProcessTime.Mean())
    
    BronxLanguageAssessorsWaitTimeAVG.append(BronxLanguageAssessorsWaitTime.Mean())
    BronxLanguageAssessorsQueueLengthAVG.append(BronxQueueLanguageAssessors.Mean())
    BronxLanguageAssessorsBusyAVG.append(BronxLanguageAssessors.Mean())
    
    BronxTeachersAdultLevelAWaitTimeAVG.append(BronxTeachersAdultLevelAWaitTime.Mean())
    BronxTeachersAdultLevelAQueueLengthAVG.append(BronxQueueTeachersAdultLevelA.Mean())
    BronxTeachersAdultLevelABusyAVG.append(BronxTeachersAdultLevelA.Mean())
    
    BronxTeachersAdultLevelBWaitTimeAVG.append(BronxTeachersAdultLevelBWaitTime.Mean())
    BronxTeachersAdultLevelBQueueLengthAVG.append(BronxQueueTeachersAdultLevelB.Mean())
    BronxTeachersAdultLevelBBusyAVG.append(BronxTeachersAdultLevelB.Mean())
    
    BronxTeachersAdultLevelCWaitTimeAVG.append(BronxTeachersAdultLevelCWaitTime.Mean())
    BronxTeachersAdultLevelCQueueLengthAVG.append(BronxQueueTeachersAdultLevelC.Mean())
    BronxTeachersAdultLevelCBusyAVG.append(BronxTeachersAdultLevelC.Mean())
    
    BronxTeachersChildLevelAWaitTimeAVG.append(BronxTeachersChildLevelAWaitTime.Mean())
    BronxTeachersChildLevelAQueueLengthAVG.append(BronxQueueTeachersChildLevelA.Mean())
    BronxTeachersChildLevelABusyAVG.append(BronxTeachersChildLevelA.Mean())
    
    BronxTeachersChildLevelBWaitTimeAVG.append(BronxTeachersChildLevelBWaitTime.Mean())
    BronxTeachersChildLevelBQueueLengthAVG.append(BronxQueueTeachersChildLevelB.Mean())
    BronxTeachersChildLevelBBusyAVG.append(BronxTeachersChildLevelB.Mean())
    
    BronxTeachersChildLevelCWaitTimeAVG.append(BronxTeachersChildLevelCWaitTime.Mean())
    BronxTeachersChildLevelCQueueLengthAVG.append(BronxQueueTeachersChildLevelC.Mean())
    BronxTeachersChildLevelCBusyAVG.append(BronxTeachersChildLevelC.Mean())
    
    BronxCareerCoachWaitTimeAVG.append(BronxCareerCoachWaitTime.Mean())
    BronxCareerCoachQueueLengthAVG.append(BronxQueueCareerCoach.Mean())
    BronxCareerCoachBusyAVG.append(BronxCareerCoach.Mean())
    
    BronxFluentChildrenTotalProcessTimeAVG.append(BronxFluentChildrenTotalProcessTime.Mean())
    BronxLevelAChildrenTotalProcessTimeAVG.append(BronxLevelAChildrenTotalProcessTime.Mean())
    BronxLevelBChildrenTotalProcessTimeAVG.append(BronxLevelBChildrenTotalProcessTime.Mean())
    BronxLevelCChildrenTotalProcessTimeAVG.append(BronxLevelCChildrenTotalProcessTime.Mean())
    
    BronxFluentEducatedAdultTotalProcessTimeAVG.append(BronxFluentEducatedAdultTotalProcessTime.Mean())
    BronxLevelAEducatedAdultTotalProcessTimeAVG.append(BronxLevelAEducatedAdultTotalProcessTime.Mean())
    BronxLevelBEducatedAdultTotalProcessTimeAVG.append(BronxLevelBEducatedAdultTotalProcessTime.Mean())
    BronxLevelCEducatedAdultTotalProcessTimeAVG.append(BronxLevelCEducatedAdultTotalProcessTime.Mean())
    BronxFluentUneducatedAdultTotalProcessTimeAVG.append(BronxFluentUneducatedAdultTotalProcessTime.Mean())
    BronxLevelAUneducatedAdultTotalProcessTimeAVG.append(BronxLevelAUneducatedAdultTotalProcessTime.Mean())
    BronxLevelBUneducatedAdultTotalProcessTimeAVG.append(BronxLevelBUneducatedAdultTotalProcessTime.Mean())
    BronxLevelCUneducatedAdultTotalProcessTimeAVG.append(BronxLevelCUneducatedAdultTotalProcessTime.Mean())
    
    BronxFluentOldTotalProcessTimeAVG.append(BronxFluentOldTotalProcessTime.Mean())
    BronxLevelAOldTotalProcessTimeAVG.append(BronxLevelAOldTotalProcessTime.Mean())
    BronxLevelBOldTotalProcessTimeAVG.append(BronxLevelBOldTotalProcessTime.Mean())
    BronxLevelCOldTotalProcessTimeAVG.append(BronxLevelCOldTotalProcessTime.Mean())
    
    BronxTotalProcessTimeAVG.append(BronxTotalProcessTime.Mean())
    
    StatenIslandLanguageAssessorsWaitTimeAVG.append(StatenIslandLanguageAssessorsWaitTime.Mean())
    StatenIslandLanguageAssessorsQueueLengthAVG.append(StatenIslandQueueLanguageAssessors.Mean())
    StatenIslandLanguageAssessorsBusyAVG.append(StatenIslandLanguageAssessors.Mean())
    
    StatenIslandTeachersAdultLevelAWaitTimeAVG.append(StatenIslandTeachersAdultLevelAWaitTime.Mean())
    StatenIslandTeachersAdultLevelAQueueLengthAVG.append(StatenIslandQueueTeachersAdultLevelA.Mean())
    StatenIslandTeachersAdultLevelABusyAVG.append(StatenIslandTeachersAdultLevelA.Mean())
    
    StatenIslandTeachersAdultLevelBWaitTimeAVG.append(StatenIslandTeachersAdultLevelBWaitTime.Mean())
    StatenIslandTeachersAdultLevelBQueueLengthAVG.append(StatenIslandQueueTeachersAdultLevelB.Mean())
    StatenIslandTeachersAdultLevelBBusyAVG.append(StatenIslandTeachersAdultLevelB.Mean())
    
    StatenIslandTeachersAdultLevelCWaitTimeAVG.append(StatenIslandTeachersAdultLevelCWaitTime.Mean())
    StatenIslandTeachersAdultLevelCQueueLengthAVG.append(StatenIslandQueueTeachersAdultLevelC.Mean())
    StatenIslandTeachersAdultLevelCBusyAVG.append(StatenIslandTeachersAdultLevelC.Mean())
    
    StatenIslandTeachersChildLevelAWaitTimeAVG.append(StatenIslandTeachersChildLevelAWaitTime.Mean())
    StatenIslandTeachersChildLevelAQueueLengthAVG.append(StatenIslandQueueTeachersChildLevelA.Mean())
    StatenIslandTeachersChildLevelABusyAVG.append(StatenIslandTeachersChildLevelA.Mean())
    
    StatenIslandTeachersChildLevelBWaitTimeAVG.append(StatenIslandTeachersChildLevelBWaitTime.Mean())
    StatenIslandTeachersChildLevelBQueueLengthAVG.append(StatenIslandQueueTeachersChildLevelB.Mean())
    StatenIslandTeachersChildLevelBBusyAVG.append(StatenIslandTeachersChildLevelB.Mean())
    
    StatenIslandTeachersChildLevelCWaitTimeAVG.append(StatenIslandTeachersChildLevelCWaitTime.Mean())
    StatenIslandTeachersChildLevelCQueueLengthAVG.append(StatenIslandQueueTeachersChildLevelC.Mean())
    StatenIslandTeachersChildLevelCBusyAVG.append(StatenIslandTeachersChildLevelC.Mean())
    
    StatenIslandCareerCoachWaitTimeAVG.append(StatenIslandCareerCoachWaitTime.Mean())
    StatenIslandCareerCoachQueueLengthAVG.append(StatenIslandQueueCareerCoach.Mean())
    StatenIslandCareerCoachBusyAVG.append(StatenIslandCareerCoach.Mean())
    
    StatenIslandFluentChildrenTotalProcessTimeAVG.append(StatenIslandFluentChildrenTotalProcessTime.Mean())
    StatenIslandLevelAChildrenTotalProcessTimeAVG.append(StatenIslandLevelAChildrenTotalProcessTime.Mean())
    StatenIslandLevelBChildrenTotalProcessTimeAVG.append(StatenIslandLevelBChildrenTotalProcessTime.Mean())
    StatenIslandLevelCChildrenTotalProcessTimeAVG.append(StatenIslandLevelCChildrenTotalProcessTime.Mean())
    
    StatenIslandFluentEducatedAdultTotalProcessTimeAVG.append(StatenIslandFluentEducatedAdultTotalProcessTime.Mean())
    StatenIslandLevelAEducatedAdultTotalProcessTimeAVG.append(StatenIslandLevelAEducatedAdultTotalProcessTime.Mean())
    StatenIslandLevelBEducatedAdultTotalProcessTimeAVG.append(StatenIslandLevelBEducatedAdultTotalProcessTime.Mean())
    StatenIslandLevelCEducatedAdultTotalProcessTimeAVG.append(StatenIslandLevelCEducatedAdultTotalProcessTime.Mean())
    StatenIslandFluentUneducatedAdultTotalProcessTimeAVG.append(StatenIslandFluentUneducatedAdultTotalProcessTime.Mean())
    StatenIslandLevelAUneducatedAdultTotalProcessTimeAVG.append(StatenIslandLevelAUneducatedAdultTotalProcessTime.Mean())
    StatenIslandLevelBUneducatedAdultTotalProcessTimeAVG.append(StatenIslandLevelBUneducatedAdultTotalProcessTime.Mean())
    StatenIslandLevelCUneducatedAdultTotalProcessTimeAVG.append(StatenIslandLevelCUneducatedAdultTotalProcessTime.Mean())
    
    StatenIslandFluentOldTotalProcessTimeAVG.append(StatenIslandFluentOldTotalProcessTime.Mean())
    StatenIslandLevelAOldTotalProcessTimeAVG.append(StatenIslandLevelAOldTotalProcessTime.Mean())
    StatenIslandLevelBOldTotalProcessTimeAVG.append(StatenIslandLevelBOldTotalProcessTime.Mean())
    StatenIslandLevelCOldTotalProcessTimeAVG.append(StatenIslandLevelCOldTotalProcessTime.Mean())
    
    StatenIslandTotalProcessTimeAVG.append(StatenIslandTotalProcessTime.Mean())
    
    Immigrants_successfully_processed_avg += Immigrants_processed_by_borough_and_age/5
    
    pbar_outer.set_description(f"Running Simulation - {rep+1}/{Epochs}")
    pbar_outer.update(1)
# Close the outer progress bar
pbar_outer.close()

Running Simulation:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration 1 in process:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration 2 in process:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration 3 in process:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration 4 in process:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration 5 in process:   0%|          | 0/2 [00:00<?, ?it/s]

In [349]:
BrooklynResultsDF = pd.DataFrame({
    "Language Assessment Average Wait Time": BrooklynLanguageAssessorsWaitTimeAVG,
    "Language Assessment Average Queue Length": BrooklynLanguageAssessorsQueueLengthAVG,
    "Language Assessors Average Utilization": np.array(BrooklynLanguageAssessorsBusyAVG)/BrooklynNumberOfLanguageAssessors,
    "Adult Level A Course Average Wait Time": BrooklynTeachersAdultLevelAWaitTimeAVG,
    "Adult Level A Course Average Queue Length": BrooklynTeachersAdultLevelAQueueLengthAVG,
    "Teachers Adult Level A Course Average Utilization": np.array(BrooklynTeachersAdultLevelABusyAVG)/BrooklynNumberOfTeachersAdultLevelA,
    "Adult Level B Course Average Wait Time": BrooklynTeachersAdultLevelBWaitTimeAVG,
    "Adult Level B Course Average Queue Length": BrooklynTeachersAdultLevelBQueueLengthAVG,
    "Teachers Adult Level B Course Average Utilization": np.array(BrooklynTeachersAdultLevelBBusyAVG)/BrooklynNumberOfTeachersAdultLevelB,
    "Adult Level C Course Average Wait Time": BrooklynTeachersAdultLevelCWaitTimeAVG,
    "Adult Level C Course Average Queue Length": BrooklynTeachersAdultLevelCQueueLengthAVG,
    "Teachers Adult Level C Course Average Utilization": np.array(BrooklynTeachersAdultLevelCBusyAVG)/BrooklynNumberOfTeachersAdultLevelC,
    "Children Level A Course Average Wait Time": BrooklynTeachersChildLevelAWaitTimeAVG,
    "Children Level A Course Average Queue Length": BrooklynTeachersChildLevelAQueueLengthAVG,
    "Teachers Children Level A Course Average Utilization": np.array(BrooklynTeachersChildLevelABusyAVG)/BrooklynNumberOfTeachersChildLevelA,
    "Children Level B Course Average Wait Time": BrooklynTeachersChildLevelBWaitTimeAVG,
    "Children Level B Course Average Queue Length": BrooklynTeachersChildLevelBQueueLengthAVG,
    "Teachers Children Level B Course Average Utilization": np.array(BrooklynTeachersChildLevelBBusyAVG)/BrooklynNumberOfTeachersChildLevelB,
    "Children Level C Course Average Wait Time": BrooklynTeachersChildLevelCWaitTimeAVG,
    "Children Level C Course Average Queue Length": BrooklynTeachersChildLevelCQueueLengthAVG,
    "Teachers Children Level C Course Average Utilization": np.array(BrooklynTeachersChildLevelCBusyAVG)/BrooklynNumberOfTeachersChildLevelC,
    "Career Guidance Average Wait Time": BrooklynCareerCoachWaitTimeAVG,
    "Career Guidance Average Queue Length": BrooklynCareerCoachQueueLengthAVG,
    "Career Guides Average Utilization": np.array(BrooklynCareerCoachBusyAVG)/BrooklynNumberOfCareerCoaches,
    "Fluent Children Total Processing Time": BrooklynFluentChildrenTotalProcessTimeAVG,
    "Level A Children Total Processing Time": BrooklynLevelAChildrenTotalProcessTimeAVG,
    "Level B Children Total Processing Time": BrooklynLevelBChildrenTotalProcessTimeAVG,
    "Level C Children Total Processing Time": BrooklynLevelCChildrenTotalProcessTimeAVG,
    "Fluent Educated Adult Total Processing Time": BrooklynFluentEducatedAdultTotalProcessTimeAVG,
    "Level A Educated Adult Total Processing Time": BrooklynLevelAEducatedAdultTotalProcessTimeAVG,
    "Level B Educated Adult Total Processing Time": BrooklynLevelBEducatedAdultTotalProcessTimeAVG,
    "Level C Educated Adult Total Processing Time": BrooklynLevelCEducatedAdultTotalProcessTimeAVG,
    "Fluent Uneducated Adult Total Processing Time": BrooklynFluentUneducatedAdultTotalProcessTimeAVG,
    "Level A Uneducated Adult Total Processing Time": BrooklynLevelAUneducatedAdultTotalProcessTimeAVG,
    "Level B Uneducated Adult Total Processing Time": BrooklynLevelBUneducatedAdultTotalProcessTimeAVG,
    "Level C Uneducated Adult Total Processing Time": BrooklynLevelCUneducatedAdultTotalProcessTimeAVG,
    "Fluent Old Total Processing Time": BrooklynFluentOldTotalProcessTimeAVG,
    "Level A Old Total Processing Time": BrooklynLevelAOldTotalProcessTimeAVG,
    "Level B Old Total Processing Time": BrooklynLevelBOldTotalProcessTimeAVG,
    "Level C Old Total Processing Time": BrooklynLevelCOldTotalProcessTimeAVG,
    "Total Average Processing Time": BrooklynTotalProcessTimeAVG
})
BrooklynResultsDF.to_csv(f"ResultsDirectory/SimulationUncooperative/BrooklynResultsMode{Mode}_{k}.csv", sep=",")
print(f"Simulation Results Saved to BrooklynResultsMode{Mode}_{k}.csv")
print("**********************************************************************************")
print("Means of the simulation results:")
print(BrooklynResultsDF.mean())
print("**********************************************************************************")
print("99% Confidence Intervals Half-Width")
CI= scs.t.ppf(1-0.01/2, len(BrooklynResultsDF)-1)*np.sqrt(BrooklynResultsDF.var()/len(BrooklynResultsDF))
print(CI)
print("**********************************************************************************")
print("Error:")
print(CI/BrooklynResultsDF.mean())
print("**********************************************************************************")
print("Number of epochs required for 99% confidence, half-width 8 (one day) for all measures:")
print(max((scs.t.ppf(1-0.01, len(BrooklynResultsDF)-1)*np.sqrt(BrooklynResultsDF.var()/len(BrooklynResultsDF))/8)**2))
print("Number of epochs required for 99% confidence, half-width 16 (two day) for all measures:")
print(max((scs.t.ppf(1-0.01/2, len(BrooklynResultsDF)-1)*np.sqrt(BrooklynResultsDF.var()/len(BrooklynResultsDF))/16)**2))

Simulation Results Saved to BrooklynResultsMode4_6.csv
**********************************************************************************
Means of the simulation results:
Language Assessment Average Wait Time                     0.000659
Language Assessment Average Queue Length                  0.010900
Language Assessors Average Utilization                    0.393790
Adult Level A Course Average Wait Time                   10.235814
Adult Level A Course Average Queue Length                38.880208
Teachers Adult Level A Course Average Utilization         0.237840
Adult Level B Course Average Wait Time                   60.260987
Adult Level B Course Average Queue Length               358.630263
Teachers Adult Level B Course Average Utilization         0.613085
Adult Level C Course Average Wait Time                  162.250443
Adult Level C Course Average Queue Length               790.071156
Teachers Adult Level C Course Average Utilization         0.834283
Children Level A Course A

In [350]:
ManhattanResultsDF = pd.DataFrame({
    "Language Assessment Average Wait Time": ManhattanLanguageAssessorsWaitTimeAVG,
    "Language Assessment Average Queue Length": ManhattanLanguageAssessorsQueueLengthAVG,
    "Language Assessors Average Utilization": np.array(ManhattanLanguageAssessorsBusyAVG)/ManhattanNumberOfLanguageAssessors,
    "Adult Level A Course Average Wait Time": ManhattanTeachersAdultLevelAWaitTimeAVG,
    "Adult Level A Course Average Queue Length": ManhattanTeachersAdultLevelAQueueLengthAVG,
    "Teachers Adult Level A Course Average Utilization": np.array(ManhattanTeachersAdultLevelABusyAVG)/ManhattanNumberOfTeachersAdultLevelA,
    "Adult Level B Course Average Wait Time": ManhattanTeachersAdultLevelBWaitTimeAVG,
    "Adult Level B Course Average Queue Length": ManhattanTeachersAdultLevelBQueueLengthAVG,
    "Teachers Adult Level B Course Average Utilization": np.array(ManhattanTeachersAdultLevelBBusyAVG)/ManhattanNumberOfTeachersAdultLevelB,
    "Adult Level C Course Average Wait Time": ManhattanTeachersAdultLevelCWaitTimeAVG,
    "Adult Level C Course Average Queue Length": ManhattanTeachersAdultLevelCQueueLengthAVG,
    "Teachers Adult Level C Course Average Utilization": np.array(ManhattanTeachersAdultLevelCBusyAVG)/ManhattanNumberOfTeachersAdultLevelC,
    "Children Level A Course Average Wait Time": ManhattanTeachersChildLevelAWaitTimeAVG,
    "Children Level A Course Average Queue Length": ManhattanTeachersChildLevelAQueueLengthAVG,
    "Teachers Children Level A Course Average Utilization": np.array(ManhattanTeachersChildLevelABusyAVG)/ManhattanNumberOfTeachersChildLevelA,
    "Children Level B Course Average Wait Time": ManhattanTeachersChildLevelBWaitTimeAVG,
    "Children Level B Course Average Queue Length": ManhattanTeachersChildLevelBQueueLengthAVG,
    "Teachers Children Level B Course Average Utilization": np.array(ManhattanTeachersChildLevelBBusyAVG)/ManhattanNumberOfTeachersChildLevelB,
    "Children Level C Course Average Wait Time": ManhattanTeachersChildLevelCWaitTimeAVG,
    "Children Level C Course Average Queue Length": ManhattanTeachersChildLevelCQueueLengthAVG,
    "Teachers Children Level C Course Average Utilization": np.array(ManhattanTeachersChildLevelCBusyAVG)/ManhattanNumberOfTeachersChildLevelC,
    "Career Guidance Average Wait Time": ManhattanCareerCoachWaitTimeAVG,
    "Career Guidance Average Queue Length": ManhattanCareerCoachQueueLengthAVG,
    "Career Guides Average Utilization": np.array(ManhattanCareerCoachBusyAVG)/ManhattanNumberOfCareerCoaches,
    "Fluent Children Total Processing Time": ManhattanFluentChildrenTotalProcessTimeAVG,
    "Level A Children Total Processing Time": ManhattanLevelAChildrenTotalProcessTimeAVG,
    "Level B Children Total Processing Time": ManhattanLevelBChildrenTotalProcessTimeAVG,
    "Level C Children Total Processing Time": ManhattanLevelCChildrenTotalProcessTimeAVG,
    "Fluent Educated Adult Total Processing Time": ManhattanFluentEducatedAdultTotalProcessTimeAVG,
    "Level A Educated Adult Total Processing Time": ManhattanLevelAEducatedAdultTotalProcessTimeAVG,
    "Level B Educated Adult Total Processing Time": ManhattanLevelBEducatedAdultTotalProcessTimeAVG,
    "Level C Educated Adult Total Processing Time": ManhattanLevelCEducatedAdultTotalProcessTimeAVG,
    "Fluent Uneducated Adult Total Processing Time": ManhattanFluentUneducatedAdultTotalProcessTimeAVG,
    "Level A Uneducated Adult Total Processing Time": ManhattanLevelAUneducatedAdultTotalProcessTimeAVG,
    "Level B Uneducated Adult Total Processing Time": ManhattanLevelBUneducatedAdultTotalProcessTimeAVG,
    "Level C Uneducated Adult Total Processing Time": ManhattanLevelCUneducatedAdultTotalProcessTimeAVG,
    "Fluent Old Total Processing Time": ManhattanFluentOldTotalProcessTimeAVG,
    "Level A Old Total Processing Time": ManhattanLevelAOldTotalProcessTimeAVG,
    "Level B Old Total Processing Time": ManhattanLevelBOldTotalProcessTimeAVG,
    "Level C Old Total Processing Time": ManhattanLevelCOldTotalProcessTimeAVG,
    "Total Average Processing Time": ManhattanTotalProcessTimeAVG
})
ManhattanResultsDF.to_csv(f"ResultsDirectory/SimulationUncooperative/ManhattanResultsMode{Mode}_{k}.csv", sep=",")
print(f"Simulation Results Saved to ManhattanResultsMode{Mode}_{k}.csv")
print("**********************************************************************************")
print("Means of the simulation results:")
print(ManhattanResultsDF.mean())
print("**********************************************************************************")
print("99% Confidence Intervals Half-Width")
CI= scs.t.ppf(1-0.01/2, len(ManhattanResultsDF)-1)*np.sqrt(ManhattanResultsDF.var()/len(ManhattanResultsDF))
print(CI)
print("**********************************************************************************")
print("Error:")
print(CI/ManhattanResultsDF.mean())
print("**********************************************************************************")
print("Number of epochs required for 99% confidence, half-width 8 (one day) for all measures:")
print(max((scs.t.ppf(1-0.01, len(ManhattanResultsDF)-1)*np.sqrt(ManhattanResultsDF.var()/len(ManhattanResultsDF))/8)**2))
print("Number of epochs required for 99% confidence, half-width 16 (two day) for all measures:")
print(max((scs.t.ppf(1-0.01/2, len(ManhattanResultsDF)-1)*np.sqrt(ManhattanResultsDF.var()/len(ManhattanResultsDF))/16)**2))

Simulation Results Saved to ManhattanResultsMode4_6.csv
**********************************************************************************
Means of the simulation results:
Language Assessment Average Wait Time                     0.009996
Language Assessment Average Queue Length                  0.079537
Language Assessors Average Utilization                    0.442010
Adult Level A Course Average Wait Time                   14.645042
Adult Level A Course Average Queue Length                26.762577
Teachers Adult Level A Course Average Utilization         0.312418
Adult Level B Course Average Wait Time                   63.001859
Adult Level B Course Average Queue Length               180.106784
Teachers Adult Level B Course Average Utilization         0.633394
Adult Level C Course Average Wait Time                  165.506694
Adult Level C Course Average Queue Length               386.873803
Teachers Adult Level C Course Average Utilization         0.846420
Children Level A Course 

In [351]:
QueensResultsDF = pd.DataFrame({
    "Language Assessment Average Wait Time": QueensLanguageAssessorsWaitTimeAVG,
    "Language Assessment Average Queue Length": QueensLanguageAssessorsQueueLengthAVG,
    "Language Assessors Average Utilization": np.array(QueensLanguageAssessorsBusyAVG)/QueensNumberOfLanguageAssessors,
    "Adult Level A Course Average Wait Time": QueensTeachersAdultLevelAWaitTimeAVG,
    "Adult Level A Course Average Queue Length": QueensTeachersAdultLevelAQueueLengthAVG,
    "Teachers Adult Level A Course Average Utilization": np.array(QueensTeachersAdultLevelABusyAVG)/QueensNumberOfTeachersAdultLevelA,
    "Adult Level B Course Average Wait Time": QueensTeachersAdultLevelBWaitTimeAVG,
    "Adult Level B Course Average Queue Length": QueensTeachersAdultLevelBQueueLengthAVG,
    "Teachers Adult Level B Course Average Utilization": np.array(QueensTeachersAdultLevelBBusyAVG)/QueensNumberOfTeachersAdultLevelB,
    "Adult Level C Course Average Wait Time": QueensTeachersAdultLevelCWaitTimeAVG,
    "Adult Level C Course Average Queue Length": QueensTeachersAdultLevelCQueueLengthAVG,
    "Teachers Adult Level C Course Average Utilization": np.array(QueensTeachersAdultLevelCBusyAVG)/QueensNumberOfTeachersAdultLevelC,
    "Children Level A Course Average Wait Time": QueensTeachersChildLevelAWaitTimeAVG,
    "Children Level A Course Average Queue Length": QueensTeachersChildLevelAQueueLengthAVG,
    "Teachers Children Level A Course Average Utilization": np.array(QueensTeachersChildLevelABusyAVG)/QueensNumberOfTeachersChildLevelA,
    "Children Level B Course Average Wait Time": QueensTeachersChildLevelBWaitTimeAVG,
    "Children Level B Course Average Queue Length": QueensTeachersChildLevelBQueueLengthAVG,
    "Teachers Children Level B Course Average Utilization": np.array(QueensTeachersChildLevelBBusyAVG)/QueensNumberOfTeachersChildLevelB,
    "Children Level C Course Average Wait Time": QueensTeachersChildLevelCWaitTimeAVG,
    "Children Level C Course Average Queue Length": QueensTeachersChildLevelCQueueLengthAVG,
    "Teachers Children Level C Course Average Utilization": np.array(QueensTeachersChildLevelCBusyAVG)/QueensNumberOfTeachersChildLevelC,
    "Career Guidance Average Wait Time": QueensCareerCoachWaitTimeAVG,
    "Career Guidance Average Queue Length": QueensCareerCoachQueueLengthAVG,
    "Career Guides Average Utilization": np.array(QueensCareerCoachBusyAVG)/QueensNumberOfCareerCoaches,
    "Fluent Children Total Processing Time": QueensFluentChildrenTotalProcessTimeAVG,
    "Level A Children Total Processing Time": QueensLevelAChildrenTotalProcessTimeAVG,
    "Level B Children Total Processing Time": QueensLevelBChildrenTotalProcessTimeAVG,
    "Level C Children Total Processing Time": QueensLevelCChildrenTotalProcessTimeAVG,
    "Fluent Educated Adult Total Processing Time": QueensFluentEducatedAdultTotalProcessTimeAVG,
    "Level A Educated Adult Total Processing Time": QueensLevelAEducatedAdultTotalProcessTimeAVG,
    "Level B Educated Adult Total Processing Time": QueensLevelBEducatedAdultTotalProcessTimeAVG,
    "Level C Educated Adult Total Processing Time": QueensLevelCEducatedAdultTotalProcessTimeAVG,
    "Fluent Uneducated Adult Total Processing Time": QueensFluentUneducatedAdultTotalProcessTimeAVG,
    "Level A Uneducated Adult Total Processing Time": QueensLevelAUneducatedAdultTotalProcessTimeAVG,
    "Level B Uneducated Adult Total Processing Time": QueensLevelBUneducatedAdultTotalProcessTimeAVG,
    "Level C Uneducated Adult Total Processing Time": QueensLevelCUneducatedAdultTotalProcessTimeAVG,
    "Fluent Old Total Processing Time": QueensFluentOldTotalProcessTimeAVG,
    "Level A Old Total Processing Time": QueensLevelAOldTotalProcessTimeAVG,
    "Level B Old Total Processing Time": QueensLevelBOldTotalProcessTimeAVG,
    "Level C Old Total Processing Time": QueensLevelCOldTotalProcessTimeAVG,
    "Total Average Processing Time": QueensTotalProcessTimeAVG
})
QueensResultsDF.to_csv(f"ResultsDirectory/SimulationUncooperative/QueensResultsMode{Mode}_{k}.csv", sep=",")
print(f"Simulation Results Saved to QueensResultsMode{Mode}_{k}.csv")
print("**********************************************************************************")
print("Means of the simulation results:")
print(QueensResultsDF.mean())
print("**********************************************************************************")
print("99% Confidence Intervals Half-Width")
CI= scs.t.ppf(1-0.01/2, len(QueensResultsDF)-1)*np.sqrt(QueensResultsDF.var()/len(QueensResultsDF))
print(CI)
print("**********************************************************************************")
print("Error:")
print(CI/QueensResultsDF.mean())
print("**********************************************************************************")
print("Number of epochs required for 99% confidence, half-width 8 (one day) for all measures:")
print(max((scs.t.ppf(1-0.01, len(QueensResultsDF)-1)*np.sqrt(QueensResultsDF.var()/len(QueensResultsDF))/8)**2))
print("Number of epochs required for 99% confidence, half-width 16 (two day) for all measures:")
print(max((scs.t.ppf(1-0.01/2, len(QueensResultsDF)-1)*np.sqrt(QueensResultsDF.var()/len(QueensResultsDF))/16)**2))

Simulation Results Saved to QueensResultsMode4_6.csv
**********************************************************************************
Means of the simulation results:
Language Assessment Average Wait Time                     0.000399
Language Assessment Average Queue Length                  0.007488
Language Assessors Average Utilization                    0.390705
Adult Level A Course Average Wait Time                    9.304376
Adult Level A Course Average Queue Length                40.020120
Teachers Adult Level A Course Average Utilization         0.220614
Adult Level B Course Average Wait Time                   62.592887
Adult Level B Course Average Queue Length               422.259075
Teachers Adult Level B Course Average Utilization         0.624921
Adult Level C Course Average Wait Time                  161.977810
Adult Level C Course Average Queue Length               893.316486
Teachers Adult Level C Course Average Utilization         0.835722
Children Level A Course Ave

In [352]:
BronxResultsDF = pd.DataFrame({
    "Language Assessment Average Wait Time": BronxLanguageAssessorsWaitTimeAVG,
    "Language Assessment Average Queue Length": BronxLanguageAssessorsQueueLengthAVG,
    "Language Assessors Average Utilization": np.array(BronxLanguageAssessorsBusyAVG)/BronxNumberOfLanguageAssessors,
    "Adult Level A Course Average Wait Time": BronxTeachersAdultLevelAWaitTimeAVG,
    "Adult Level A Course Average Queue Length": BronxTeachersAdultLevelAQueueLengthAVG,
    "Teachers Adult Level A Course Average Utilization": np.array(BronxTeachersAdultLevelABusyAVG)/BronxNumberOfTeachersAdultLevelA,
    "Adult Level B Course Average Wait Time": BronxTeachersAdultLevelBWaitTimeAVG,
    "Adult Level B Course Average Queue Length": BronxTeachersAdultLevelBQueueLengthAVG,
    "Teachers Adult Level B Course Average Utilization": np.array(BronxTeachersAdultLevelBBusyAVG)/BronxNumberOfTeachersAdultLevelB,
    "Adult Level C Course Average Wait Time": BronxTeachersAdultLevelCWaitTimeAVG,
    "Adult Level C Course Average Queue Length": BronxTeachersAdultLevelCQueueLengthAVG,
    "Teachers Adult Level C Course Average Utilization": np.array(BronxTeachersAdultLevelCBusyAVG)/BronxNumberOfTeachersAdultLevelC,
    "Children Level A Course Average Wait Time": BronxTeachersChildLevelAWaitTimeAVG,
    "Children Level A Course Average Queue Length": BronxTeachersChildLevelAQueueLengthAVG,
    "Teachers Children Level A Course Average Utilization": np.array(BronxTeachersChildLevelABusyAVG)/BronxNumberOfTeachersChildLevelA,
    "Children Level B Course Average Wait Time": BronxTeachersChildLevelBWaitTimeAVG,
    "Children Level B Course Average Queue Length": BronxTeachersChildLevelBQueueLengthAVG,
    "Teachers Children Level B Course Average Utilization": np.array(BronxTeachersChildLevelBBusyAVG)/BronxNumberOfTeachersChildLevelB,
    "Children Level C Course Average Wait Time": BronxTeachersChildLevelCWaitTimeAVG,
    "Children Level C Course Average Queue Length": BronxTeachersChildLevelCQueueLengthAVG,
    "Teachers Children Level C Course Average Utilization": np.array(BronxTeachersChildLevelCBusyAVG)/BronxNumberOfTeachersChildLevelC,
    "Career Guidance Average Wait Time": BronxCareerCoachWaitTimeAVG,
    "Career Guidance Average Queue Length": BronxCareerCoachQueueLengthAVG,
    "Career Guides Average Utilization": np.array(BronxCareerCoachBusyAVG)/BronxNumberOfCareerCoaches,
    "Fluent Children Total Processing Time": BronxFluentChildrenTotalProcessTimeAVG,
    "Level A Children Total Processing Time": BronxLevelAChildrenTotalProcessTimeAVG,
    "Level B Children Total Processing Time": BronxLevelBChildrenTotalProcessTimeAVG,
    "Level C Children Total Processing Time": BronxLevelCChildrenTotalProcessTimeAVG,
    "Fluent Educated Adult Total Processing Time": BronxFluentEducatedAdultTotalProcessTimeAVG,
    "Level A Educated Adult Total Processing Time": BronxLevelAEducatedAdultTotalProcessTimeAVG,
    "Level B Educated Adult Total Processing Time": BronxLevelBEducatedAdultTotalProcessTimeAVG,
    "Level C Educated Adult Total Processing Time": BronxLevelCEducatedAdultTotalProcessTimeAVG,
    "Fluent Uneducated Adult Total Processing Time": BronxFluentUneducatedAdultTotalProcessTimeAVG,
    "Level A Uneducated Adult Total Processing Time": BronxLevelAUneducatedAdultTotalProcessTimeAVG,
    "Level B Uneducated Adult Total Processing Time": BronxLevelBUneducatedAdultTotalProcessTimeAVG,
    "Level C Uneducated Adult Total Processing Time": BronxLevelCUneducatedAdultTotalProcessTimeAVG,
    "Fluent Old Total Processing Time": BronxFluentOldTotalProcessTimeAVG,
    "Level A Old Total Processing Time": BronxLevelAOldTotalProcessTimeAVG,
    "Level B Old Total Processing Time": BronxLevelBOldTotalProcessTimeAVG,
    "Level C Old Total Processing Time": BronxLevelCOldTotalProcessTimeAVG,
    "Total Average Processing Time": BronxTotalProcessTimeAVG
})
BronxResultsDF.to_csv(f"ResultsDirectory/SimulationUncooperative/BronxResultsMode{Mode}_{k}.csv", sep=",")
print(f"Simulation Results Saved to BronxResultsMode{Mode}_{k}.csv")
print("**********************************************************************************")
print("Means of the simulation results:")
print(BronxResultsDF.mean())
print("**********************************************************************************")
print("99% Confidence Intervals Half-Width")
CI= scs.t.ppf(1-0.01/2, len(BronxResultsDF)-1)*np.sqrt(BronxResultsDF.var()/len(BronxResultsDF))
print(CI)
print("**********************************************************************************")
print("Error:")
print(CI/BronxResultsDF.mean())
print("**********************************************************************************")
print("Number of epochs required for 99% confidence, half-width 8 (one day) for all measures:")
print(max((scs.t.ppf(1-0.01, len(BronxResultsDF)-1)*np.sqrt(BronxResultsDF.var()/len(BronxResultsDF))/8)**2))
print("Number of epochs required for 99% confidence, half-width 16 (two day) for all measures:")
print(max((scs.t.ppf(1-0.01/2, len(BronxResultsDF)-1)*np.sqrt(BronxResultsDF.var()/len(BronxResultsDF))/16)**2))

Simulation Results Saved to BronxResultsMode4_6.csv
**********************************************************************************
Means of the simulation results:
Language Assessment Average Wait Time                     0.002554
Language Assessment Average Queue Length                  0.021761
Language Assessors Average Utilization                    0.354975
Adult Level A Course Average Wait Time                   11.830166
Adult Level A Course Average Queue Length                23.169756
Teachers Adult Level A Course Average Utilization         0.244305
Adult Level B Course Average Wait Time                   68.547243
Adult Level B Course Average Queue Length               209.607294
Teachers Adult Level B Course Average Utilization         0.661430
Adult Level C Course Average Wait Time                  171.340042
Adult Level C Course Average Queue Length               429.654118
Teachers Adult Level C Course Average Utilization         0.855947
Children Level A Course Aver

In [353]:
StatenIslandResultsDF = pd.DataFrame({
    "Language Assessment Average Wait Time": StatenIslandLanguageAssessorsWaitTimeAVG,
    "Language Assessment Average Queue Length": StatenIslandLanguageAssessorsQueueLengthAVG,
    "Language Assessors Average Utilization": np.array(StatenIslandLanguageAssessorsBusyAVG)/StatenIslandNumberOfLanguageAssessors,
    "Adult Level A Course Average Wait Time": StatenIslandTeachersAdultLevelAWaitTimeAVG,
    "Adult Level A Course Average Queue Length": StatenIslandTeachersAdultLevelAQueueLengthAVG,
    "Teachers Adult Level A Course Average Utilization": np.array(StatenIslandTeachersAdultLevelABusyAVG)/StatenIslandNumberOfTeachersAdultLevelA,
    "Adult Level B Course Average Wait Time": StatenIslandTeachersAdultLevelBWaitTimeAVG,
    "Adult Level B Course Average Queue Length": StatenIslandTeachersAdultLevelBQueueLengthAVG,
    "Teachers Adult Level B Course Average Utilization": np.array(StatenIslandTeachersAdultLevelBBusyAVG)/StatenIslandNumberOfTeachersAdultLevelB,
    "Adult Level C Course Average Wait Time": StatenIslandTeachersAdultLevelCWaitTimeAVG,
    "Adult Level C Course Average Queue Length": StatenIslandTeachersAdultLevelCQueueLengthAVG,
    "Teachers Adult Level C Course Average Utilization": np.array(StatenIslandTeachersAdultLevelCBusyAVG)/StatenIslandNumberOfTeachersAdultLevelC,
    "Children Level A Course Average Wait Time": StatenIslandTeachersChildLevelAWaitTimeAVG,
    "Children Level A Course Average Queue Length": StatenIslandTeachersChildLevelAQueueLengthAVG,
    "Teachers Children Level A Course Average Utilization": np.array(StatenIslandTeachersChildLevelABusyAVG)/StatenIslandNumberOfTeachersChildLevelA,
    "Children Level B Course Average Wait Time": StatenIslandTeachersChildLevelBWaitTimeAVG,
    "Children Level B Course Average Queue Length": StatenIslandTeachersChildLevelBQueueLengthAVG,
    "Teachers Children Level B Course Average Utilization": np.array(StatenIslandTeachersChildLevelBBusyAVG)/StatenIslandNumberOfTeachersChildLevelB,
    "Children Level C Course Average Wait Time": StatenIslandTeachersChildLevelCWaitTimeAVG,
    "Children Level C Course Average Queue Length": StatenIslandTeachersChildLevelCQueueLengthAVG,
    "Teachers Children Level C Course Average Utilization": np.array(StatenIslandTeachersChildLevelCBusyAVG)/StatenIslandNumberOfTeachersChildLevelC,
    "Career Guidance Average Wait Time": StatenIslandCareerCoachWaitTimeAVG,
    "Career Guidance Average Queue Length": StatenIslandCareerCoachQueueLengthAVG,
    "Career Guides Average Utilization": np.array(StatenIslandCareerCoachBusyAVG)/StatenIslandNumberOfCareerCoaches,
    "Fluent Children Total Processing Time": StatenIslandFluentChildrenTotalProcessTimeAVG,
    "Level A Children Total Processing Time": StatenIslandLevelAChildrenTotalProcessTimeAVG,
    "Level B Children Total Processing Time": StatenIslandLevelBChildrenTotalProcessTimeAVG,
    "Level C Children Total Processing Time": StatenIslandLevelCChildrenTotalProcessTimeAVG,
    "Fluent Educated Adult Total Processing Time": StatenIslandFluentEducatedAdultTotalProcessTimeAVG,
    "Level A Educated Adult Total Processing Time": StatenIslandLevelAEducatedAdultTotalProcessTimeAVG,
    "Level B Educated Adult Total Processing Time": StatenIslandLevelBEducatedAdultTotalProcessTimeAVG,
    "Level C Educated Adult Total Processing Time": StatenIslandLevelCEducatedAdultTotalProcessTimeAVG,
    "Fluent Uneducated Adult Total Processing Time": StatenIslandFluentUneducatedAdultTotalProcessTimeAVG,
    "Level A Uneducated Adult Total Processing Time": StatenIslandLevelAUneducatedAdultTotalProcessTimeAVG,
    "Level B Uneducated Adult Total Processing Time": StatenIslandLevelBUneducatedAdultTotalProcessTimeAVG,
    "Level C Uneducated Adult Total Processing Time": StatenIslandLevelCUneducatedAdultTotalProcessTimeAVG,
    "Fluent Old Total Processing Time": StatenIslandFluentOldTotalProcessTimeAVG,
    "Level A Old Total Processing Time": StatenIslandLevelAOldTotalProcessTimeAVG,
    "Level B Old Total Processing Time": StatenIslandLevelBOldTotalProcessTimeAVG,
    "Level C Old Total Processing Time": StatenIslandLevelCOldTotalProcessTimeAVG,
    "Total Average Processing Time": StatenIslandTotalProcessTimeAVG
})
StatenIslandResultsDF.to_csv(f"ResultsDirectory/SimulationUncooperative/StatenIslandResultsMode{Mode}_{k}.csv", sep=",")
print(f"Simulation Results Saved to StatenIslandResultsMode{Mode}_{k}.csv")
print("**********************************************************************************")
print("Means of the simulation results:")
print(StatenIslandResultsDF.mean())
print("**********************************************************************************")
print("99% Confidence Intervals Half-Width")
CI= scs.t.ppf(1-0.01/2, len(StatenIslandResultsDF)-1)*np.sqrt(StatenIslandResultsDF.var()/len(StatenIslandResultsDF))
print(CI)
print("**********************************************************************************")
print("Error:")
print(CI/StatenIslandResultsDF.mean())
print("**********************************************************************************")
print("Number of epochs required for 99% confidence, half-width 8 (one day) for all measures:")
print(max((scs.t.ppf(1-0.01, len(StatenIslandResultsDF)-1)*np.sqrt(StatenIslandResultsDF.var()/len(StatenIslandResultsDF))/8)**2))
print("Number of epochs required for 99% confidence, half-width 16 (two day) for all measures:")
print(max((scs.t.ppf(1-0.01/2, len(StatenIslandResultsDF)-1)*np.sqrt(StatenIslandResultsDF.var()/len(StatenIslandResultsDF))/16)**2))

Simulation Results Saved to StatenIslandResultsMode4_6.csv
**********************************************************************************
Means of the simulation results:
Language Assessment Average Wait Time                      0.033907
Language Assessment Average Queue Length                   0.058252
Language Assessors Average Utilization                     0.286257
Adult Level A Course Average Wait Time                    44.993793
Adult Level A Course Average Queue Length                 17.705445
Teachers Adult Level A Course Average Utilization          0.511018
Adult Level B Course Average Wait Time                   124.740944
Adult Level B Course Average Queue Length                 76.187190
Teachers Adult Level B Course Average Utilization          0.854827
Adult Level C Course Average Wait Time                   215.263505
Adult Level C Course Average Queue Length                105.586253
Teachers Adult Level C Course Average Utilization          0.896572
Children 

In [354]:
Brooklyn_N = max((scs.t.ppf(1-0.01/2, len(BrooklynResultsDF)-1)*np.sqrt(BrooklynResultsDF.var()/len(BrooklynResultsDF))/16**2))
Manhattan_N = max((scs.t.ppf(1-0.01/2, len(ManhattanResultsDF)-1)*np.sqrt(ManhattanResultsDF.var()/len(ManhattanResultsDF))/16)**2)
Queens_N = max((scs.t.ppf(1-0.01/2, len(QueensResultsDF)-1)*np.sqrt(QueensResultsDF.var()/len(QueensResultsDF))/16)**2)
Bronx_N = max((scs.t.ppf(1-0.01/2, len(BronxResultsDF)-1)*np.sqrt(BronxResultsDF.var()/len(BronxResultsDF))/16)**2)
StatenIsland_N = max((scs.t.ppf(1-0.01/2, len(StatenIslandResultsDF)-1)*np.sqrt(StatenIslandResultsDF.var()/len(StatenIslandResultsDF))/16)**2)

print(f"Number of epochs required for 99% confidence, half-width 16 (two days) for all measures: {math.ceil(max(Brooklyn_N, Manhattan_N, Queens_N, Bronx_N, StatenIsland_N))}")

Number of epochs required for 99% confidence, half-width 16 (two days) for all measures: 99



# According to a single pilot (5 epochs) we would require 65 iterations to achieve confidence intervals with confidence level of 99\% and half width of 16 (2 days) for all tracked statistics


In [355]:
QueueLengthsDict = {
    "BrooklynLanguageAssessors": BrooklynLanguageAssessorsQueueLength,
    "BrooklynTeachersAdultLevelA": BrooklynTeachersAdultLevelAQueueLength,
    "BrooklynTeachersAdultLevelB": BrooklynTeachersAdultLevelBQueueLength,
    "BrooklynTeachersAdultLevelC": BrooklynTeachersAdultLevelCQueueLength,
    "BrooklynTeachersChildLevelA": BrooklynTeachersChildLevelAQueueLength,
    "BrooklynTeachersChildLevelB": BrooklynTeachersChildLevelBQueueLength,
    "BrooklynTeachersChildLevelC": BrooklynTeachersChildLevelCQueueLength,
    "BrooklynCareerGuides": BrooklynCareerCoachesQueueLength,
    "ManhattanLanguageAssessors": ManhattanLanguageAssessorsQueueLength,
    "ManhattanTeachersAdultLevelA": ManhattanTeachersAdultLevelAQueueLength,
    "ManhattanTeachersAdultLevelB": ManhattanTeachersAdultLevelBQueueLength,
    "ManhattanTeachersAdultLevelC": ManhattanTeachersAdultLevelCQueueLength,
    "ManhattanTeachersChildLevelA": ManhattanTeachersChildLevelAQueueLength,
    "ManhattanTeachersChildLevelB": ManhattanTeachersChildLevelBQueueLength,
    "ManhattanTeachersChildLevelC": ManhattanTeachersChildLevelCQueueLength,
    "ManhattanCareerGuides": ManhattanCareerCoachesQueueLength,
    "QueensLanguageAssessors": QueensLanguageAssessorsQueueLength,
    "QueensTeachersAdultLevelA": QueensTeachersAdultLevelAQueueLength,
    "QueensTeachersAdultLevelB": QueensTeachersAdultLevelBQueueLength,
    "QueensTeachersAdultLevelC": QueensTeachersAdultLevelCQueueLength,
    "QueensTeachersChildLevelA": QueensTeachersChildLevelAQueueLength,
    "QueensTeachersChildLevelB": QueensTeachersChildLevelBQueueLength,
    "QueensTeachersChildLevelC": QueensTeachersChildLevelCQueueLength,
    "QueensCareerGuides": QueensCareerCoachesQueueLength,
    "BronxLanguageAssessors": BronxLanguageAssessorsQueueLength,
    "BronxTeachersAdultLevelA": BronxTeachersAdultLevelAQueueLength,
    "BronxTeachersAdultLevelB": BronxTeachersAdultLevelBQueueLength,
    "BronxTeachersAdultLevelC": BronxTeachersAdultLevelCQueueLength,
    "BronxTeachersChildLevelA": BronxTeachersChildLevelAQueueLength,
    "BronxTeachersChildLevelB": BronxTeachersChildLevelBQueueLength,
    "BronxTeachersChildLevelC": BronxTeachersChildLevelCQueueLength,
    "BronxCareerGuides": BronxCareerCoachesQueueLength,
    "StatenIslandLanguageAssessors": StatenIslandLanguageAssessorsQueueLength,
    "StatenIslandTeachersAdultLevelA": StatenIslandTeachersAdultLevelAQueueLength,
    "StatenIslandTeachersAdultLevelB": StatenIslandTeachersAdultLevelBQueueLength,
    "StatenIslandTeachersAdultLevelC": StatenIslandTeachersAdultLevelCQueueLength,
    "StatenIslandTeachersChildLevelA": StatenIslandTeachersChildLevelAQueueLength,
    "StatenIslandTeachersChildLevelB": StatenIslandTeachersChildLevelBQueueLength,
    "StatenIslandTeachersChildLevelC": StatenIslandTeachersChildLevelCQueueLength,
    "StatenIslandCareerGuides": StatenIslandCareerCoachesQueueLength
}
#Adding zeros to force into df. didnt have time to find a better alternative:(
max_length = max(len(arr) for arr in QueueLengthsDict.values())
for key, value in QueueLengthsDict.items():
    QueueLengthsDict[key] += [0] * (max_length - len(value))
QueueLengthsDataFrame = pd.DataFrame(QueueLengthsDict)
QueueLengthsDataFrame.to_csv(f"ResultsDirectory/SimulationUncooperative/QueueLengthsMode{Mode}_{k}.csv", sep=",")
print(f"Simulation Results Saved to QueueLengthsMode{Mode}_{k}.csv")

Simulation Results Saved to QueueLengthsMode4_6.csv


In [356]:
Immigrants_successfully_processed_avg_df = pd.DataFrame(Immigrants_successfully_processed_avg, columns = ['Children', 'Adults', 'Old'])
Immigrants_successfully_processed_avg_df.to_csv(f"ResultsDirectory/SimulationUncooperative/ImmigrantsProcessedMode{Mode}_{k}.csv", sep=",")